In [ ]:
#pip install selenium
#To scrape the unique ids

In [1]:
import json
import pandas as pd

from bs4 import BeautifulSoup
from requests import get
import requests

from selenium import webdriver
import time

from random import randint
from time import sleep

import pprint
pp = pprint.PrettyPrinter(indent =4)

In [ ]:
#This is to flatten nested list
import itertools

def oneDArray(x):
    return list(itertools.chain(*x))

In [2]:
Review = pd.read_json('reviews_Beauty_5.json',lines = True)

In [3]:
Review.head(100)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
95,ASXPNU896V9I8,B00004U9UY,L90,"[0, 0]","I have dry hands, as my job requires me to was...",5,Best hand lotion,1360800000,"02 14, 2013"
96,A3S3R88HA0HZG3,B00004U9UY,PT Cruiser,"[1, 1]",This hand cream has one of the nicest fragranc...,5,An effective hand cream with a wonderful scent,1238544000,"04 1, 2009"
97,A2SX3JU39YE2TP,B00004U9UY,Summer Haven,"[0, 0]",I first purchased this hand treatment from a g...,4,"Wonderful feel, wonderful fragrance",1374192000,"07 19, 2013"
98,AD526SECC3I9D,B00004U9V2,Anuradha J. Cetta,"[0, 0]","Best hand cream I've ever used, and I NEVER us...",5,Great cream!,1386201600,"12 5, 2013"


In [4]:
type(Review)

pandas.core.frame.DataFrame

# EDA and Scraping


In [5]:
#Number of data points
Review.shape
#There are 198502 data points

(198502, 9)

In [6]:
#Finding the unique products
len(Review['asin'].unique())
#There are 12101 unique products

12101

In [7]:
#Checking the number of unique users
len(Review['reviewerID'].unique())
#There are 22363 unique user

22363

In [8]:
#columns
Review.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [9]:
Review.head(50)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
5,A3BTN14HIZET6Z,7806397051,"S. M. Randall ""WildHorseWoman""","[1, 2]","I was very happy to get this palette, now I wi...",5,Very nice palette!,1365984000,"04 15, 2013"
6,A1Z59RFKN0M5QL,7806397051,"tasha ""luvely12b""","[1, 3]",PLEASE DONT DO IT! this just rachett the palet...,1,smh!!!,1376611200,"08 16, 2013"
7,AWUO9P6PL1SY8,7806397051,TreMagnifique,"[0, 1]","Chalky,Not Pigmented,Wears off easily,Not a Co...",2,"Chalky, Not Pigmented, Wears off easily, Not a...",1378252800,"09 4, 2013"
8,A3LMILRM9OC3SA,9759091062,NaN,"[0, 0]",Did nothing for me. Stings when I put it on. I...,2,"no Lightening, no Brightening,......NOTHING",1405209600,"07 13, 2014"
9,A30IP88QK3YUIO,9759091062,Amina Bint Ibraheem,"[0, 0]",I bought this product to get rid of the dark s...,3,Its alright,1388102400,"12 27, 2013"


The columns are:
1. reviewerID: This column gives the unique identity to each amazon user.
2 asin: This column gives the product number
3. helpful: List L, where L[0]: Number of people who found it helpful, L[1]: Number of people who did not find it helpful
4.reviewText: Complete descriotion of the review
5.overall: overall rating of the product
6.Summary: brief into to the review
7.unixReviewTime: Time of review
8. reviewTime: Date of Review

The asin column will be used to scrape additional data from the current amazon product website
A list containing unique asin id is formed which will be used to iterate during web scraping


In [10]:
unique_asin =[]
unique_asin = list(Review['asin'].unique())

In [11]:
type(unique_asin)

list

In [13]:
unique_asin[:10]

['7806397051',
 '9759091062',
 '9788072216',
 '9790790961',
 '9790794231',
 'B00004TMFE',
 'B00004TUBL',
 'B00004TUBV',
 'B00004U9UY',
 'B00004U9V2']

The unique_asin list will be iterated through to enter the product page and scrape for more information. Due to block from anti robot pages, fields with unavailable rows will be rescraped using a brand new driver object and then visiting the same url again. 
The url of the product page is of the format: https://www.amazon.com/gp/product/{asin id}

# # #Web Scraping

In [12]:
#The file did not have updated product name, color and description, we will scrape it from amazon website
import selenium
from selenium import webdriver

In [17]:
#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")

In [18]:
#Creating an empty list to store dictionary for each product
list_of_products = []

The unique_asin list is first iterated and tags are located to scrape the following data
1. Product name
2. Price
3. Rating
4. Number of Rating
5. Category of beauty product
6. Availability

In [19]:
#Instantiating the web driver for product with asin B00004TMFE
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
driver.get(f'https://www.amazon.com/gp/product/B00004TMFE')

# Instantiate BeautifulSoup web scraper 
soup = BeautifulSoup(driver.page_source, 'lxml') 

#dictionary to store the key-value pair
prod1_dict = {}

# Save product ID 
prod1_dict['asin'] = 'B00004TMFE'

# Look for product title 
prod1_dict['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
# Look for product categories list 
prod1_dict['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
        
# Look for product description
desc = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
prod1_dict['description'] =" ".join(desc.split())
        
#Look for Product Price
prod1_dict['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
#Look for Rating
prod1_dict['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
#Number of Ratings
prod1_dict['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()

pp.pprint(prod1_dict)
                            

{   'No_of_Rating': '5,646 ratings',
    'asin': 'B00004TMFE',
    'category': 'Hair Care',
    'description': 'Brand Avalon Organics Scent Biotin B Hair Type Thin Liquid '
                   'Volume 14 Fluid Ounces Ingredients Aloe Barbadensis Leaf '
                   'Juice (Certified Organic Ingredient), Aqua (Water), '
                   'Brassica Alcohol, Glycerin, Cocos Nucifera (Coconut) Oil '
                   '(Certified Organic Ingredient), Brassicyl Isoleucinate '
                   'Esylate, Gluconolactone, Capryloyl Glycerin/Sebacic Acid '
                   'Copolymer, Cedrus Atlantica Wood Oil, Citrus Aurantifolia '
                   '(Lime) Oil, Citrus Aurantium Dulcis (Orange) Peel Oil, '
                   'Citrus Limon (Lemon) Peel Oil, Citrus Paradisi '
                   '(Grapefruit) Peel Oil, Eucalyptus Globulus Leaf Oil, '
                   'Juniperus Virginiana Oil, Persea Gratissima (Avocado) Oil '
                   '(Certified Organic Ingredient), Rosmar

# Scraping First 10 Products

In [63]:
list_of_products_10 = []
list_of_unavailable_products_10 =[]

#Instantiating the web driver(product 1-10)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")

#Start looping through first 500 products in the asin list
for i in unique_asin[:10]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
        
        # Look for product description
        desc = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        product_info['description']=" ".join(desc.split())
        
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10.append(product_info)
        

    except:
        list_of_unavailable_products_10.append(i)
        print('not available',i)
        

https://www.amazon.com/gp/product/7806397051
not available 7806397051
https://www.amazon.com/gp/product/9759091062
not available 9759091062
https://www.amazon.com/gp/product/9788072216
adding to dictionary 9788072216
https://www.amazon.com/gp/product/9790790961
not available 9790790961
https://www.amazon.com/gp/product/9790794231
not available 9790794231
https://www.amazon.com/gp/product/B00004TMFE
adding to dictionary B00004TMFE
https://www.amazon.com/gp/product/B00004TUBL
adding to dictionary B00004TUBL
https://www.amazon.com/gp/product/B00004TUBV
adding to dictionary B00004TUBV
https://www.amazon.com/gp/product/B00004U9UY
not available B00004U9UY
https://www.amazon.com/gp/product/B00004U9V2
not available B00004U9V2


In [64]:
list_of_unavailable_products_10

['7806397051',
 '9759091062',
 '9790790961',
 '9790794231',
 'B00004U9UY',
 'B00004U9V2']

In [22]:
print(len(list_of_products_10))
print(len(list_of_unavailable_products_10))

4
6


In [65]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_10 = pd.DataFrame(list_of_products_10)

list_of_products_10.to_csv('list_of_products_10.csv')

In [66]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_10 = pd.DataFrame(list_of_unavailable_products_10)

list_of_unavailable_products_10.to_csv('list_of_unavailable_products_10.csv')

In [67]:
list_of_unavailable_products_10=list_of_unavailable_products_10.values.tolist()

In [72]:
list_of_unavailable_products_10

[['7806397051'],
 ['9759091062'],
 ['9790790961'],
 ['9790794231'],
 ['B00004U9UY'],
 ['B00004U9V2']]

In [74]:
oneDArray(list_of_unavailable_products_10)

['7806397051',
 '9759091062',
 '9790790961',
 '9790794231',
 'B00004U9UY',
 'B00004U9V2']

In [25]:
#!cat 'list_of_products_10.csv'

In [26]:
pp.pprint(list_of_products_10)

         asin                                               name   category  \
0  9788072216  Prada Candy by Prada for Women 1.7 oz Eau de P...  Fragrance   
1  B00004TMFE  Avalon Organics Therapy Thickening Conditioner...  Hair Care   
2  B00004TUBL  CLASSIC Better Living Two Chamber Dispenser, W...       Bath   
3  B00004TUBV  Better Living Products 73350 ULTI-MATE 3 Chamb...       Bath   

                                         description   price  \
0  Brand Prada Scent Honey, Musk , Vanilla Item F...  $56.29   
1  Brand Avalon Organics Scent Biotin B Hair Type...   $7.81   
2  Color White Brand CLASSIC Item Dimensions LxWx...  $39.99   
3  Color White Material Plastic Brand Better Livi...  $38.21   

               rating   No_of_Rating  
0  4.7 out of 5 stars  1,391 ratings  
1  4.1 out of 5 stars  5,646 ratings  
2  4.3 out of 5 stars     99 ratings  
3  4.5 out of 5 stars  1,245 ratings  


# #Scrape Products 1-2000

In [27]:
list_of_products_2000 = []
list_of_unavailable_products_2000 =[]

#Instantiating the web driver(product 1-500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")

#Start looping through first 500 products in the asin list
for i in unique_asin[:500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
       
        # Look for product description
        desc = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        product_info['description']=" ".join(desc.split())
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 500-1000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")


#Start looping through 500-1000 products in the asin list
for i in unique_asin[500:1000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 1000-15000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 1000-1500 products in the asin list
for i in unique_asin[1000:1500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 1500-2000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 1000-1500 products in the asin list
for i in unique_asin[1500:2000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_2000.append(product_info)
        

    except:
        list_of_unavailable_products_2000.append(i)
        print('not available',i)


        

https://www.amazon.com/gp/product/7806397051
not available 7806397051
https://www.amazon.com/gp/product/9759091062
not available 9759091062
https://www.amazon.com/gp/product/9788072216
adding to dictionary 9788072216
https://www.amazon.com/gp/product/9790790961
not available 9790790961
https://www.amazon.com/gp/product/9790794231
not available 9790794231
https://www.amazon.com/gp/product/B00004TMFE
adding to dictionary B00004TMFE
https://www.amazon.com/gp/product/B00004TUBL
adding to dictionary B00004TUBL
https://www.amazon.com/gp/product/B00004TUBV
adding to dictionary B00004TUBV
https://www.amazon.com/gp/product/B00004U9UY
not available B00004U9UY
https://www.amazon.com/gp/product/B00004U9V2
not available B00004U9V2
https://www.amazon.com/gp/product/B000050B6U
not available B000050B6U
https://www.amazon.com/gp/product/B000052WY7
not available B000052WY7
https://www.amazon.com/gp/product/B000052WYD
adding to dictionary B000052WYD
https://www.amazon.com/gp/product/B000052WYL
not availa

adding to dictionary B0000536P4
https://www.amazon.com/gp/product/B00005375C
adding to dictionary B00005375C
https://www.amazon.com/gp/product/B0000537NH
not available B0000537NH
https://www.amazon.com/gp/product/B0000537SD
not available B0000537SD
https://www.amazon.com/gp/product/B000053L6W
adding to dictionary B000053L6W
https://www.amazon.com/gp/product/B000056I89
adding to dictionary B000056I89
https://www.amazon.com/gp/product/B000056W5Y
not available B000056W5Y
https://www.amazon.com/gp/product/B000056W74
adding to dictionary B000056W74
https://www.amazon.com/gp/product/B00005A43X
adding to dictionary B00005A43X
https://www.amazon.com/gp/product/B00005A443
adding to dictionary B00005A443
https://www.amazon.com/gp/product/B00005A9WP
not available B00005A9WP
https://www.amazon.com/gp/product/B00005AVC7
not available B00005AVC7
https://www.amazon.com/gp/product/B00005B9FV
adding to dictionary B00005B9FV
https://www.amazon.com/gp/product/B00005CDSP
not available B00005CDSP
https://w

adding to dictionary B0000Y3FQC
https://www.amazon.com/gp/product/B0000Y3FRG
not available B0000Y3FRG
https://www.amazon.com/gp/product/B0000Y3GF2
adding to dictionary B0000Y3GF2
https://www.amazon.com/gp/product/B0000Y3GIE
adding to dictionary B0000Y3GIE
https://www.amazon.com/gp/product/B0000Y3LAC
adding to dictionary B0000Y3LAC
https://www.amazon.com/gp/product/B0000Y8H3S
adding to dictionary B0000Y8H3S
https://www.amazon.com/gp/product/B0000YUWY0
not available B0000YUWY0
https://www.amazon.com/gp/product/B0000YUX4O
not available B0000YUX4O
https://www.amazon.com/gp/product/B0000YUXGW
adding to dictionary B0000YUXGW
https://www.amazon.com/gp/product/B0000YUXI0
adding to dictionary B0000YUXI0
https://www.amazon.com/gp/product/B0000YV6BI
adding to dictionary B0000YV6BI
https://www.amazon.com/gp/product/B0000YV6YK
adding to dictionary B0000YV6YK
https://www.amazon.com/gp/product/B0000YV83O
not available B0000YV83O
https://www.amazon.com/gp/product/B0000ZHH5G
adding to dictionary B0000Z

adding to dictionary B0001AGMWE
https://www.amazon.com/gp/product/B0001AQCII
not available B0001AQCII
https://www.amazon.com/gp/product/B0001B86HM
adding to dictionary B0001B86HM
https://www.amazon.com/gp/product/B0001EKTZQ
adding to dictionary B0001EKTZQ
https://www.amazon.com/gp/product/B0001EKWPI
not available B0001EKWPI
https://www.amazon.com/gp/product/B0001EKYL0
not available B0001EKYL0
https://www.amazon.com/gp/product/B0001EL0CW
not available B0001EL0CW
https://www.amazon.com/gp/product/B0001EL5R2
adding to dictionary B0001EL5R2
https://www.amazon.com/gp/product/B0001EL658
not available B0001EL658
https://www.amazon.com/gp/product/B0001FF80G
not available B0001FF80G
https://www.amazon.com/gp/product/B0001I8ZT4
adding to dictionary B0001I8ZT4
https://www.amazon.com/gp/product/B0001LH6YG
adding to dictionary B0001LH6YG
https://www.amazon.com/gp/product/B0001M3K4U
not available B0001M3K4U
https://www.amazon.com/gp/product/B0001M7D7A
adding to dictionary B0001M7D7A
https://www.amaz

not available B00028LV4Y
https://www.amazon.com/gp/product/B00028M3N2
adding to dictionary B00028M3N2
https://www.amazon.com/gp/product/B00028MLG6
not available B00028MLG6
https://www.amazon.com/gp/product/B00028MLHA
adding to dictionary B00028MLHA
https://www.amazon.com/gp/product/B00028O7ZO
adding to dictionary B00028O7ZO
https://www.amazon.com/gp/product/B00028OC4K
adding to dictionary B00028OC4K
https://www.amazon.com/gp/product/B00028OPZ6
adding to dictionary B00028OPZ6
https://www.amazon.com/gp/product/B00028OQZA
not available B00028OQZA
https://www.amazon.com/gp/product/B00028OSI0
adding to dictionary B00028OSI0
https://www.amazon.com/gp/product/B00028OTAM
adding to dictionary B00028OTAM
https://www.amazon.com/gp/product/B00028OTNY
not available B00028OTNY
https://www.amazon.com/gp/product/B00028OU5Q
not available B00028OU5Q
https://www.amazon.com/gp/product/B00028PDWK
adding to dictionary B00028PDWK
https://www.amazon.com/gp/product/B00028PFF0
adding to dictionary B00028PFF0
ht

...

adding to dictionary B000H3CPNU
https://www.amazon.com/gp/product/B000H3CPO4
adding to dictionary B000H3CPO4
https://www.amazon.com/gp/product/B000H3EMMM
not available B000H3EMMM
https://www.amazon.com/gp/product/B000H3EMPO
not available B000H3EMPO
https://www.amazon.com/gp/product/B000H3ENH6
not available B000H3ENH6
https://www.amazon.com/gp/product/B000H3EO34
adding to dictionary B000H3EO34
https://www.amazon.com/gp/product/B000H3EO9I
adding to dictionary B000H3EO9I
https://www.amazon.com/gp/product/B000H3EOLG
adding to dictionary B000H3EOLG
https://www.amazon.com/gp/product/B000H3GCXY
not available B000H3GCXY
https://www.amazon.com/gp/product/B000H3GDZQ
not available B000H3GDZQ
https://www.amazon.com/gp/product/B000H3I0JS
not available B000H3I0JS
https://www.amazon.com/gp/product/B000H3I30O
not available B000H3I30O
https://www.amazon.com/gp/product/B000H3I33G
not available B000H3I33G
https://www.amazon.com/gp/product/B000H3I3Y0
adding to dictionary B000H3I3Y0
https://www.amazon.com/

adding to dictionary B000JFFTDE
https://www.amazon.com/gp/product/B000JG1QDA
adding to dictionary B000JG1QDA
https://www.amazon.com/gp/product/B000JGERAO
not available B000JGERAO
https://www.amazon.com/gp/product/B000JGKG2M
not available B000JGKG2M
https://www.amazon.com/gp/product/B000JI9JYQ
adding to dictionary B000JI9JYQ
https://www.amazon.com/gp/product/B000JIN0OG
not available B000JIN0OG
https://www.amazon.com/gp/product/B000JKEB38
not available B000JKEB38
https://www.amazon.com/gp/product/B000JKIE4A
not available B000JKIE4A
https://www.amazon.com/gp/product/B000JKN4X6
not available B000JKN4X6
https://www.amazon.com/gp/product/B000JKPDQ2
not available B000JKPDQ2
https://www.amazon.com/gp/product/B000JL3U1Q
adding to dictionary B000JL3U1Q
https://www.amazon.com/gp/product/B000JL3WNW
not available B000JL3WNW
https://www.amazon.com/gp/product/B000JL5V0E
adding to dictionary B000JL5V0E
https://www.amazon.com/gp/product/B000JL69OG
adding to dictionary B000JL69OG
https://www.amazon.com/

adding to dictionary B000LVW25O
https://www.amazon.com/gp/product/B000LW1I2Q
not available B000LW1I2Q
https://www.amazon.com/gp/product/B000LXTNMW
not available B000LXTNMW
https://www.amazon.com/gp/product/B000LY3X7W
adding to dictionary B000LY3X7W
https://www.amazon.com/gp/product/B000M4AAWW
adding to dictionary B000M4AAWW
https://www.amazon.com/gp/product/B000M8BE3C
not available B000M8BE3C
https://www.amazon.com/gp/product/B000MANVP4
not available B000MANVP4
https://www.amazon.com/gp/product/B000MANVQ8
not available B000MANVQ8
https://www.amazon.com/gp/product/B000MANVXG
not available B000MANVXG
https://www.amazon.com/gp/product/B000MAOWO8
adding to dictionary B000MAOWO8
https://www.amazon.com/gp/product/B000MAOWOS
adding to dictionary B000MAOWOS
https://www.amazon.com/gp/product/B000MAOWP2
adding to dictionary B000MAOWP2
https://www.amazon.com/gp/product/B000MAQWA0
adding to dictionary B000MAQWA0
https://www.amazon.com/gp/product/B000MASTM4
adding to dictionary B000MASTM4
https://w

not available B000NKHNI4
https://www.amazon.com/gp/product/B000NKL3D0
adding to dictionary B000NKL3D0
https://www.amazon.com/gp/product/B000NL34JU
not available B000NL34JU
https://www.amazon.com/gp/product/B000NLHXEC
not available B000NLHXEC
https://www.amazon.com/gp/product/B000NLJFNY
not available B000NLJFNY
https://www.amazon.com/gp/product/B000NMHIF0
not available B000NMHIF0
https://www.amazon.com/gp/product/B000NN3H86
adding to dictionary B000NN3H86
https://www.amazon.com/gp/product/B000NN5HFW
not available B000NN5HFW
https://www.amazon.com/gp/product/B000NN5HK2
not available B000NN5HK2
https://www.amazon.com/gp/product/B000NN7FSO
adding to dictionary B000NN7FSO
https://www.amazon.com/gp/product/B000NNCA16
adding to dictionary B000NNCA16
https://www.amazon.com/gp/product/B000NNCJLM
adding to dictionary B000NNCJLM
https://www.amazon.com/gp/product/B000NNDNYY
not available B000NNDNYY
https://www.amazon.com/gp/product/B000NNG8Z0
adding to dictionary B000NNG8Z0
https://www.amazon.com/

not available B000ORM4EE
https://www.amazon.com/gp/product/B000ORORQ2
not available B000ORORQ2
https://www.amazon.com/gp/product/B000ORV3NC
adding to dictionary B000ORV3NC
https://www.amazon.com/gp/product/B000ORV5HQ
not available B000ORV5HQ
https://www.amazon.com/gp/product/B000ORXJ3E
not available B000ORXJ3E
https://www.amazon.com/gp/product/B000OUJ04S
not available B000OUJ04S
https://www.amazon.com/gp/product/B000OUTTP8
adding to dictionary B000OUTTP8
https://www.amazon.com/gp/product/B000OUY7ZA
not available B000OUY7ZA
https://www.amazon.com/gp/product/B000OV3ZDO
not available B000OV3ZDO
https://www.amazon.com/gp/product/B000OV5PEG
not available B000OV5PEG
https://www.amazon.com/gp/product/B000OW9QCW
adding to dictionary B000OW9QCW
https://www.amazon.com/gp/product/B000OWHKMU
not available B000OWHKMU
https://www.amazon.com/gp/product/B000OY3WNO
adding to dictionary B000OY3WNO
https://www.amazon.com/gp/product/B000OY5W3M
adding to dictionary B000OY5W3M
https://www.amazon.com/gp/prod

In [28]:
print(len(list_of_products_2000))
print(len(list_of_unavailable_products_2000))

846
1154


In [29]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_2000 = pd.DataFrame(list_of_products_2000)

list_of_products_2000.to_csv('list_of_products_2000.csv')

In [30]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_2000 = pd.DataFrame(list_of_unavailable_products_2000)

list_of_unavailable_products_2000.to_csv('list_of_unavailable_products_2000.csv')

## Scrape Products 2000-4000 

In [31]:
list_of_products_4000 = []
list_of_unavailable_products_4000 =[]


#Instantiating the web driver(product 2000-2500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 2000-2500 products in the asin list
for i in unique_asin[2000:2500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 2500-3000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 2500-3000 products in the asin list
for i in unique_asin[2500:3000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 3000-35000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 3000-3500 products in the asin list
for i in unique_asin[3000:3500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 3500-4000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 3500-4000 products in the asin list
for i in unique_asin[3500:4000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_4000.append(product_info)
        

    except:
        list_of_unavailable_products_4000.append(i)
        print('not available',i)
        

https://www.amazon.com/gp/product/B000PHP86E
adding to dictionary B000PHP86E
https://www.amazon.com/gp/product/B000PHP8L4
adding to dictionary B000PHP8L4
https://www.amazon.com/gp/product/B000PHRIQM
not available B000PHRIQM
https://www.amazon.com/gp/product/B000PHRIYE
adding to dictionary B000PHRIYE
https://www.amazon.com/gp/product/B000PHUKEE
adding to dictionary B000PHUKEE
https://www.amazon.com/gp/product/B000PHVNE0
not available B000PHVNE0
https://www.amazon.com/gp/product/B000PHVTVC
adding to dictionary B000PHVTVC
https://www.amazon.com/gp/product/B000PHVU8O
adding to dictionary B000PHVU8O
https://www.amazon.com/gp/product/B000PHZ26K
not available B000PHZ26K
https://www.amazon.com/gp/product/B000PHZE7M
not available B000PHZE7M
https://www.amazon.com/gp/product/B000PI36W6
adding to dictionary B000PI36W6
https://www.amazon.com/gp/product/B000PI3BS0
not available B000PI3BS0
https://www.amazon.com/gp/product/B000PI4J3G
not available B000PI4J3G
https://www.amazon.com/gp/product/B000PI5

adding to dictionary B000R80ZTQ
https://www.amazon.com/gp/product/B000R810A4
not available B000R810A4
https://www.amazon.com/gp/product/B000R830IE
adding to dictionary B000R830IE
https://www.amazon.com/gp/product/B000R858B6
not available B000R858B6
https://www.amazon.com/gp/product/B000R871A2
not available B000R871A2
https://www.amazon.com/gp/product/B000R87B1Q
adding to dictionary B000R87B1Q
https://www.amazon.com/gp/product/B000R8I8I6
not available B000R8I8I6
https://www.amazon.com/gp/product/B000R8PC9E
not available B000R8PC9E
https://www.amazon.com/gp/product/B000R8QV4Y
not available B000R8QV4Y
https://www.amazon.com/gp/product/B000R8U1XQ
adding to dictionary B000R8U1XQ
https://www.amazon.com/gp/product/B000R8UET2
not available B000R8UET2
https://www.amazon.com/gp/product/B000R8WL02
not available B000R8WL02
https://www.amazon.com/gp/product/B000R8WOT0
not available B000R8WOT0
https://www.amazon.com/gp/product/B000R8ZAMI
adding to dictionary B000R8ZAMI
https://www.amazon.com/gp/prod

adding to dictionary B000TDBJHQ
https://www.amazon.com/gp/product/B000TDFA3K
adding to dictionary B000TDFA3K
https://www.amazon.com/gp/product/B000TDFJZY
adding to dictionary B000TDFJZY
https://www.amazon.com/gp/product/B000TDG2PU
adding to dictionary B000TDG2PU
https://www.amazon.com/gp/product/B000TF70J0
adding to dictionary B000TF70J0
https://www.amazon.com/gp/product/B000TFADXA
adding to dictionary B000TFADXA
https://www.amazon.com/gp/product/B000TFJHZ0
adding to dictionary B000TFJHZ0
https://www.amazon.com/gp/product/B000TFPGP0
adding to dictionary B000TFPGP0
https://www.amazon.com/gp/product/B000TFQH5I
adding to dictionary B000TFQH5I
https://www.amazon.com/gp/product/B000TG850M
adding to dictionary B000TG850M
https://www.amazon.com/gp/product/B000TGCJWC
not available B000TGCJWC
https://www.amazon.com/gp/product/B000TGEJT8
adding to dictionary B000TGEJT8
https://www.amazon.com/gp/product/B000THCPDO
not available B000THCPDO
https://www.amazon.com/gp/product/B000THCTOO
adding to dic

not available B000V2F9RQ
https://www.amazon.com/gp/product/B000V2FB88
adding to dictionary B000V2FB88
https://www.amazon.com/gp/product/B000V2J3ZA
not available B000V2J3ZA
https://www.amazon.com/gp/product/B000V2J3ZK
not available B000V2J3ZK
https://www.amazon.com/gp/product/B000V2OZ6M
adding to dictionary B000V2OZ6M
https://www.amazon.com/gp/product/B000V2R3Y8
adding to dictionary B000V2R3Y8
https://www.amazon.com/gp/product/B000V321BM
not available B000V321BM
https://www.amazon.com/gp/product/B000V33OGS
adding to dictionary B000V33OGS
https://www.amazon.com/gp/product/B000V36SZW
not available B000V36SZW
https://www.amazon.com/gp/product/B000V3MRTI
not available B000V3MRTI
https://www.amazon.com/gp/product/B000V3RZ04
not available B000V3RZ04
https://www.amazon.com/gp/product/B000V4PL22
adding to dictionary B000V4PL22
https://www.amazon.com/gp/product/B000V500YU
adding to dictionary B000V500YU
https://www.amazon.com/gp/product/B000V58ANS
not available B000V58ANS
https://www.amazon.com/

adding to dictionary B000VWP3GS
https://www.amazon.com/gp/product/B000VWPQ9C
not available B000VWPQ9C
https://www.amazon.com/gp/product/B000VWSER8
not available B000VWSER8
https://www.amazon.com/gp/product/B000VWSKJU
not available B000VWSKJU
https://www.amazon.com/gp/product/B000VXPAPQ
not available B000VXPAPQ
https://www.amazon.com/gp/product/B000VXUUR4
not available B000VXUUR4
https://www.amazon.com/gp/product/B000VXVEGA
not available B000VXVEGA
https://www.amazon.com/gp/product/B000VY2ZAI
not available B000VY2ZAI
https://www.amazon.com/gp/product/B000VYHG9S
not available B000VYHG9S
https://www.amazon.com/gp/product/B000VYIH3M
adding to dictionary B000VYIH3M
https://www.amazon.com/gp/product/B000VZ9UFU
not available B000VZ9UFU
https://www.amazon.com/gp/product/B000VZ9UI2
not available B000VZ9UI2
https://www.amazon.com/gp/product/B000VZDWB8
not available B000VZDWB8
https://www.amazon.com/gp/product/B000VZS3VW
not available B000VZS3VW
https://www.amazon.com/gp/product/B000W0C07Y
not av

...

not available B001BOFIRO
https://www.amazon.com/gp/product/B001BOZXME
adding to dictionary B001BOZXME
https://www.amazon.com/gp/product/B001BPFQF2
adding to dictionary B001BPFQF2
https://www.amazon.com/gp/product/B001BQXTQO
not available B001BQXTQO
https://www.amazon.com/gp/product/B001BR97A0
adding to dictionary B001BR97A0
https://www.amazon.com/gp/product/B001BSH2E2
not available B001BSH2E2
https://www.amazon.com/gp/product/B001BSJ2PO
not available B001BSJ2PO
https://www.amazon.com/gp/product/B001BSJ7P4
not available B001BSJ7P4
https://www.amazon.com/gp/product/B001BVGJPC
not available B001BVGJPC
https://www.amazon.com/gp/product/B001C1LBG8
not available B001C1LBG8
https://www.amazon.com/gp/product/B001C2E10A
not available B001C2E10A
https://www.amazon.com/gp/product/B001C2NF5M
not available B001C2NF5M
https://www.amazon.com/gp/product/B001C2WQNY
not available B001C2WQNY
https://www.amazon.com/gp/product/B001C3HF20
adding to dictionary B001C3HF20
https://www.amazon.com/gp/product/B00

not available B001DK4HRS
https://www.amazon.com/gp/product/B001DK9RQE
not available B001DK9RQE
https://www.amazon.com/gp/product/B001DKEW6O
not available B001DKEW6O
https://www.amazon.com/gp/product/B001DKF08S
not available B001DKF08S
https://www.amazon.com/gp/product/B001DKGJQU
not available B001DKGJQU
https://www.amazon.com/gp/product/B001DKIFSA
not available B001DKIFSA
https://www.amazon.com/gp/product/B001DKK7Y0
not available B001DKK7Y0
https://www.amazon.com/gp/product/B001DKO9JO
adding to dictionary B001DKO9JO
https://www.amazon.com/gp/product/B001DKQ3O8
adding to dictionary B001DKQ3O8
https://www.amazon.com/gp/product/B001DKRM8O
not available B001DKRM8O
https://www.amazon.com/gp/product/B001DKRMB6
not available B001DKRMB6
https://www.amazon.com/gp/product/B001DKVBJU
adding to dictionary B001DKVBJU
https://www.amazon.com/gp/product/B001DMPAVI
not available B001DMPAVI
https://www.amazon.com/gp/product/B001DNG6XS
adding to dictionary B001DNG6XS
https://www.amazon.com/gp/product/B00

not available B001ECHLBY
https://www.amazon.com/gp/product/B001ECQ3A4
not available B001ECQ3A4
https://www.amazon.com/gp/product/B001ECQ3YU
adding to dictionary B001ECQ3YU
https://www.amazon.com/gp/product/B001ECQ4IU
adding to dictionary B001ECQ4IU
https://www.amazon.com/gp/product/B001ECQ4JE
adding to dictionary B001ECQ4JE
https://www.amazon.com/gp/product/B001ECQ4L2
not available B001ECQ4L2
https://www.amazon.com/gp/product/B001ECQ4OY
not available B001ECQ4OY
https://www.amazon.com/gp/product/B001ECQ4PS
not available B001ECQ4PS
https://www.amazon.com/gp/product/B001ECQ4R6
adding to dictionary B001ECQ4R6
https://www.amazon.com/gp/product/B001ECQ4RG
adding to dictionary B001ECQ4RG
https://www.amazon.com/gp/product/B001ECQ4RQ
not available B001ECQ4RQ
https://www.amazon.com/gp/product/B001ECQ4TO
not available B001ECQ4TO
https://www.amazon.com/gp/product/B001ECQ4TY
not available B001ECQ4TY
https://www.amazon.com/gp/product/B001ECQ4WQ
adding to dictionary B001ECQ4WQ
https://www.amazon.com/

not available B001ET7M98
https://www.amazon.com/gp/product/B001ET7MJI
not available B001ET7MJI
https://www.amazon.com/gp/product/B001EUS64M
adding to dictionary B001EUS64M
https://www.amazon.com/gp/product/B001EUS8G8
not available B001EUS8G8
https://www.amazon.com/gp/product/B001EVUXY2
not available B001EVUXY2
https://www.amazon.com/gp/product/B001EWEP7C
not available B001EWEP7C
https://www.amazon.com/gp/product/B001EWET8W
not available B001EWET8W
https://www.amazon.com/gp/product/B001EWET96
not available B001EWET96
https://www.amazon.com/gp/product/B001EWETAK
adding to dictionary B001EWETAK
https://www.amazon.com/gp/product/B001EWF2F6
not available B001EWF2F6
https://www.amazon.com/gp/product/B001EWF2H4
not available B001EWF2H4
https://www.amazon.com/gp/product/B001EWF2SI
adding to dictionary B001EWF2SI
https://www.amazon.com/gp/product/B001EWF2TM
not available B001EWF2TM
https://www.amazon.com/gp/product/B001EWF3DC
not available B001EWF3DC
https://www.amazon.com/gp/product/B001EWF3F0

adding to dictionary B001G7POOI
https://www.amazon.com/gp/product/B001G7PPNI
adding to dictionary B001G7PPNI
https://www.amazon.com/gp/product/B001G7PQKA
adding to dictionary B001G7PQKA
https://www.amazon.com/gp/product/B001G7PSCQ
not available B001G7PSCQ
https://www.amazon.com/gp/product/B001G7PU4M
not available B001G7PU4M
https://www.amazon.com/gp/product/B001G7PWAE
not available B001G7PWAE
https://www.amazon.com/gp/product/B001G7PWCM
not available B001G7PWCM
https://www.amazon.com/gp/product/B001G7PWD6
not available B001G7PWD6
https://www.amazon.com/gp/product/B001G7PWI6
not available B001G7PWI6
https://www.amazon.com/gp/product/B001G7PZRY
not available B001G7PZRY
https://www.amazon.com/gp/product/B001G7QYZQ
not available B001G7QYZQ
https://www.amazon.com/gp/product/B001G7R2UM
not available B001G7R2UM
https://www.amazon.com/gp/product/B001G7R3JW
adding to dictionary B001G7R3JW
https://www.amazon.com/gp/product/B001G7R3QK
not available B001G7R3QK
https://www.amazon.com/gp/product/B00

In [32]:
print(len(list_of_products_4000))
print(len(list_of_unavailable_products_4000))

802
1198


In [33]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_4000 = pd.DataFrame(list_of_products_4000)

list_of_products_4000.to_csv('list_of_products_4000.csv')

In [34]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_4000 = pd.DataFrame(list_of_unavailable_products_4000)

list_of_unavailable_products_4000.to_csv('list_of_unavailable_products_4000.csv')

# #Scrape Products 4000-6000 

In [35]:
list_of_products_6000 = []
list_of_unavailable_products_6000 =[]

#Instantiating the web driver(product 4000-4500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 4000-4500 products in the asin list
for i in unique_asin[4000:4500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 4500-5000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 4500-5000 products in the asin list
for i in unique_asin[4500:5000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 5000-55000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 5000-5500 products in the asin list
for i in unique_asin[5000:5500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 5500-6000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 5500-6000 products in the asin list
for i in unique_asin[5500:6000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_6000.append(product_info)
        

    except:
        list_of_unavailable_products_6000.append(i)
        print('not available',i)

     

https://www.amazon.com/gp/product/B001HTCYPC
not available B001HTCYPC
https://www.amazon.com/gp/product/B001HTTBWQ
adding to dictionary B001HTTBWQ
https://www.amazon.com/gp/product/B001HTYJLO
adding to dictionary B001HTYJLO
https://www.amazon.com/gp/product/B001HU6DZS
not available B001HU6DZS
https://www.amazon.com/gp/product/B001HYHUY2
adding to dictionary B001HYHUY2
https://www.amazon.com/gp/product/B001I124CC
not available B001I124CC
https://www.amazon.com/gp/product/B001I2DL00
not available B001I2DL00
https://www.amazon.com/gp/product/B001I45ZFC
adding to dictionary B001I45ZFC
https://www.amazon.com/gp/product/B001I6NZ0W
not available B001I6NZ0W
https://www.amazon.com/gp/product/B001I6RW6U
not available B001I6RW6U
https://www.amazon.com/gp/product/B001I6U028
not available B001I6U028
https://www.amazon.com/gp/product/B001I90NM2
not available B001I90NM2
https://www.amazon.com/gp/product/B001I90NN6
not available B001I90NN6
https://www.amazon.com/gp/product/B001I90NNQ
not available B00

not available B001KYS1YM
https://www.amazon.com/gp/product/B001KYS87M
not available B001KYS87M
https://www.amazon.com/gp/product/B001KYS930
adding to dictionary B001KYS930
https://www.amazon.com/gp/product/B001KYTWZ4
adding to dictionary B001KYTWZ4
https://www.amazon.com/gp/product/B001KYU1H2
adding to dictionary B001KYU1H2
https://www.amazon.com/gp/product/B001KYU5LE
not available B001KYU5LE
https://www.amazon.com/gp/product/B001KYU78U
adding to dictionary B001KYU78U
https://www.amazon.com/gp/product/B001KYU89S
not available B001KYU89S
https://www.amazon.com/gp/product/B001KYUHGM
not available B001KYUHGM
https://www.amazon.com/gp/product/B001KYVKL8
not available B001KYVKL8
https://www.amazon.com/gp/product/B001KYVL2G
not available B001KYVL2G
https://www.amazon.com/gp/product/B001KYVLSU
not available B001KYVLSU
https://www.amazon.com/gp/product/B001KYVOJQ
adding to dictionary B001KYVOJQ
https://www.amazon.com/gp/product/B001KYVPGI
adding to dictionary B001KYVPGI
https://www.amazon.com/

adding to dictionary B001MW38KY
https://www.amazon.com/gp/product/B001MW4VXW
not available B001MW4VXW
https://www.amazon.com/gp/product/B001MWV40U
not available B001MWV40U
https://www.amazon.com/gp/product/B001MX89Y8
adding to dictionary B001MX89Y8
https://www.amazon.com/gp/product/B001MXKG6M
not available B001MXKG6M
https://www.amazon.com/gp/product/B001MXW67Y
not available B001MXW67Y
https://www.amazon.com/gp/product/B001N2OGK4
adding to dictionary B001N2OGK4
https://www.amazon.com/gp/product/B001N3FBZW
not available B001N3FBZW
https://www.amazon.com/gp/product/B001N444EK
adding to dictionary B001N444EK
https://www.amazon.com/gp/product/B001N444H2
adding to dictionary B001N444H2
https://www.amazon.com/gp/product/B001N444HW
not available B001N444HW
https://www.amazon.com/gp/product/B001N444JK
adding to dictionary B001N444JK
https://www.amazon.com/gp/product/B001N444KY
not available B001N444KY
https://www.amazon.com/gp/product/B001N5FRWW
not available B001N5FRWW
https://www.amazon.com/

not available B001QD0OYM
https://www.amazon.com/gp/product/B001QD4RUY
not available B001QD4RUY
https://www.amazon.com/gp/product/B001QDDJB2
not available B001QDDJB2
https://www.amazon.com/gp/product/B001QF3GHM
not available B001QF3GHM
https://www.amazon.com/gp/product/B001QFDTWO
adding to dictionary B001QFDTWO
https://www.amazon.com/gp/product/B001QLACDW
not available B001QLACDW
https://www.amazon.com/gp/product/B001QNA4QU
not available B001QNA4QU
https://www.amazon.com/gp/product/B001QNG3I8
not available B001QNG3I8
https://www.amazon.com/gp/product/B001QOGXO6
not available B001QOGXO6
https://www.amazon.com/gp/product/B001QOGXOG
not available B001QOGXOG
https://www.amazon.com/gp/product/B001QOGXP0
not available B001QOGXP0
https://www.amazon.com/gp/product/B001QOP2KM
adding to dictionary B001QOP2KM
https://www.amazon.com/gp/product/B001QT05S6
adding to dictionary B001QT05S6
https://www.amazon.com/gp/product/B001QT3GOQ
adding to dictionary B001QT3GOQ
https://www.amazon.com/gp/product/B00

adding to dictionary B001T8S7T8
https://www.amazon.com/gp/product/B001T8S928
adding to dictionary B001T8S928
https://www.amazon.com/gp/product/B001T8SBDK
adding to dictionary B001T8SBDK
https://www.amazon.com/gp/product/B001T8UFPM
adding to dictionary B001T8UFPM
https://www.amazon.com/gp/product/B001T8W4R4
not available B001T8W4R4
https://www.amazon.com/gp/product/B001T8W5EQ
adding to dictionary B001T8W5EQ
https://www.amazon.com/gp/product/B001T8W5LO
not available B001T8W5LO
https://www.amazon.com/gp/product/B001TAIFZM
not available B001TAIFZM
https://www.amazon.com/gp/product/B001TBH4ZI
adding to dictionary B001TBH4ZI
https://www.amazon.com/gp/product/B001TH1480
not available B001TH1480
https://www.amazon.com/gp/product/B001TI8XBU
adding to dictionary B001TI8XBU
https://www.amazon.com/gp/product/B001TIP24Q
not available B001TIP24Q
https://www.amazon.com/gp/product/B001TJXI5U
adding to dictionary B001TJXI5U
https://www.amazon.com/gp/product/B001TKTXYE
not available B001TKTXYE
https://w

...

adding to dictionary B002Z3R7F6
https://www.amazon.com/gp/product/B002Z64W5Q
not available B002Z64W5Q
https://www.amazon.com/gp/product/B002Z684Q4
not available B002Z684Q4
https://www.amazon.com/gp/product/B002Z7EOOE
not available B002Z7EOOE
https://www.amazon.com/gp/product/B002Z7FVTQ
adding to dictionary B002Z7FVTQ
https://www.amazon.com/gp/product/B002ZASTP6
adding to dictionary B002ZASTP6
https://www.amazon.com/gp/product/B002ZBBX4O
not available B002ZBBX4O
https://www.amazon.com/gp/product/B002ZG94UY
not available B002ZG94UY
https://www.amazon.com/gp/product/B002ZG96HA
adding to dictionary B002ZG96HA
https://www.amazon.com/gp/product/B002ZHR33I
not available B002ZHR33I
https://www.amazon.com/gp/product/B002ZJ1L84
not available B002ZJ1L84
https://www.amazon.com/gp/product/B002ZNJI8K
adding to dictionary B002ZNJI8K
https://www.amazon.com/gp/product/B002ZNJI94
not available B002ZNJI94
https://www.amazon.com/gp/product/B002ZNJI9Y
adding to dictionary B002ZNJI9Y
https://www.amazon.com/

not available B0032ZQJ1E
https://www.amazon.com/gp/product/B0033041GI
adding to dictionary B0033041GI
https://www.amazon.com/gp/product/B00331YTGY
not available B00331YTGY
https://www.amazon.com/gp/product/B0033AYPXC
not available B0033AYPXC
https://www.amazon.com/gp/product/B0033EUIO8
adding to dictionary B0033EUIO8
https://www.amazon.com/gp/product/B0033EWRSI
not available B0033EWRSI
https://www.amazon.com/gp/product/B0033GLYQM
adding to dictionary B0033GLYQM
https://www.amazon.com/gp/product/B0033H00ZW
adding to dictionary B0033H00ZW
https://www.amazon.com/gp/product/B0033I9Y1C
adding to dictionary B0033I9Y1C
https://www.amazon.com/gp/product/B0033PSM3G
not available B0033PSM3G
https://www.amazon.com/gp/product/B0033PSM4K
adding to dictionary B0033PSM4K
https://www.amazon.com/gp/product/B0033PSM6S
not available B0033PSM6S
https://www.amazon.com/gp/product/B0033RGU1A
not available B0033RGU1A
https://www.amazon.com/gp/product/B0033ROOS6
not available B0033ROOS6
https://www.amazon.com/

not available B0036QTJDO
https://www.amazon.com/gp/product/B0036QYAAQ
not available B0036QYAAQ
https://www.amazon.com/gp/product/B0036RFMCA
adding to dictionary B0036RFMCA
https://www.amazon.com/gp/product/B0036UMM0M
not available B0036UMM0M
https://www.amazon.com/gp/product/B0036UW5VS
not available B0036UW5VS
https://www.amazon.com/gp/product/B0036UXD4Q
not available B0036UXD4Q
https://www.amazon.com/gp/product/B0036WJ456
not available B0036WJ456
https://www.amazon.com/gp/product/B0036WTTBK
not available B0036WTTBK
https://www.amazon.com/gp/product/B0036Z0HL8
not available B0036Z0HL8
https://www.amazon.com/gp/product/B0036Z469M
not available B0036Z469M
https://www.amazon.com/gp/product/B003708ZBQ
adding to dictionary B003708ZBQ
https://www.amazon.com/gp/product/B003709CUE
adding to dictionary B003709CUE
https://www.amazon.com/gp/product/B00371PJ1E
adding to dictionary B00371PJ1E
https://www.amazon.com/gp/product/B00373DIYM
not available B00373DIYM
https://www.amazon.com/gp/product/B00

adding to dictionary B0039BCT56
https://www.amazon.com/gp/product/B0039BCT88
not available B0039BCT88
https://www.amazon.com/gp/product/B0039BGHW2
adding to dictionary B0039BGHW2
https://www.amazon.com/gp/product/B0039BGHX6
adding to dictionary B0039BGHX6
https://www.amazon.com/gp/product/B0039I4ENY
not available B0039I4ENY
https://www.amazon.com/gp/product/B0039JYPVO
not available B0039JYPVO
https://www.amazon.com/gp/product/B0039KOL1W
not available B0039KOL1W
https://www.amazon.com/gp/product/B0039LZW78
not available B0039LZW78
https://www.amazon.com/gp/product/B0039NMCPG
not available B0039NMCPG
https://www.amazon.com/gp/product/B0039OQ0FI
adding to dictionary B0039OQ0FI
https://www.amazon.com/gp/product/B0039OVDJ6
adding to dictionary B0039OVDJ6
https://www.amazon.com/gp/product/B0039PZQ2U
adding to dictionary B0039PZQ2U
https://www.amazon.com/gp/product/B0039UT7DE
adding to dictionary B0039UT7DE
https://www.amazon.com/gp/product/B0039UT9QE
not available B0039UT9QE
https://www.amaz

not available B003CIJE4U
https://www.amazon.com/gp/product/B003CIZQGA
not available B003CIZQGA
https://www.amazon.com/gp/product/B003COAFPQ
not available B003COAFPQ
https://www.amazon.com/gp/product/B003CQZ1A8
not available B003CQZ1A8
https://www.amazon.com/gp/product/B003CT35B2
not available B003CT35B2
https://www.amazon.com/gp/product/B003CTU5MO
adding to dictionary B003CTU5MO
https://www.amazon.com/gp/product/B003CTVSKC
adding to dictionary B003CTVSKC
https://www.amazon.com/gp/product/B003CW9VCG
not available B003CW9VCG
https://www.amazon.com/gp/product/B003CX2F96
not available B003CX2F96
https://www.amazon.com/gp/product/B003CYDA9O
adding to dictionary B003CYDA9O
https://www.amazon.com/gp/product/B003CYFC18
adding to dictionary B003CYFC18
https://www.amazon.com/gp/product/B003CYUS26
adding to dictionary B003CYUS26
https://www.amazon.com/gp/product/B003CYWTQY
adding to dictionary B003CYWTQY
https://www.amazon.com/gp/product/B003D1YOCS
not available B003D1YOCS
https://www.amazon.com/

In [36]:
print(len(list_of_products_6000))
print(len(list_of_unavailable_products_6000))

781
1219


In [37]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_6000 = pd.DataFrame(list_of_products_6000)

list_of_products_6000.to_csv('list_of_products_6000.csv')

In [38]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_6000 = pd.DataFrame(list_of_unavailable_products_6000)

list_of_unavailable_products_6000.to_csv('list_of_unavailable_products_6000.csv')

# Scrape Products 6000-8000

In [39]:
list_of_products_8000 = []
list_of_unavailable_products_8000 =[]


#Instantiating the web driver(product 6000-6500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 6000-6500 products in the asin list
for i in unique_asin[6000:6500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 6500-7000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 6500-7000 products in the asin list
for i in unique_asin[6500:7000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 7000-75000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 7000-7500 products in the asin list
for i in unique_asin[7000:7500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 7500-8000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 7500-8000 products in the asin list
for i in unique_asin[7500:8000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_8000.append(product_info)
        

    except:
        list_of_unavailable_products_8000.append(i)
        print('not available',i)

        

https://www.amazon.com/gp/product/B003FITQGI
adding to dictionary B003FITQGI
https://www.amazon.com/gp/product/B003FKSSII
adding to dictionary B003FKSSII
https://www.amazon.com/gp/product/B003FKSVMG
adding to dictionary B003FKSVMG
https://www.amazon.com/gp/product/B003FKUW3M
not available B003FKUW3M
https://www.amazon.com/gp/product/B003FO70Z6
adding to dictionary B003FO70Z6
https://www.amazon.com/gp/product/B003FODWVC
adding to dictionary B003FODWVC
https://www.amazon.com/gp/product/B003FOG1JW
adding to dictionary B003FOG1JW
https://www.amazon.com/gp/product/B003FP86SK
not available B003FP86SK
https://www.amazon.com/gp/product/B003FPE7MY
not available B003FPE7MY
https://www.amazon.com/gp/product/B003FS3L9G
not available B003FS3L9G
https://www.amazon.com/gp/product/B003FSGRW4
not available B003FSGRW4
https://www.amazon.com/gp/product/B003FTXLZY
not available B003FTXLZY
https://www.amazon.com/gp/product/B003FV82VA
adding to dictionary B003FV82VA
https://www.amazon.com/gp/product/B003FXA

adding to dictionary B003JA3KSW
https://www.amazon.com/gp/product/B003JA5OJA
not available B003JA5OJA
https://www.amazon.com/gp/product/B003JBGMZ4
not available B003JBGMZ4
https://www.amazon.com/gp/product/B003JCLP6E
not available B003JCLP6E
https://www.amazon.com/gp/product/B003JFBXMM
adding to dictionary B003JFBXMM
https://www.amazon.com/gp/product/B003JKHUR4
adding to dictionary B003JKHUR4
https://www.amazon.com/gp/product/B003JLV8TY
not available B003JLV8TY
https://www.amazon.com/gp/product/B003JMWFK4
not available B003JMWFK4
https://www.amazon.com/gp/product/B003JMYAAW
adding to dictionary B003JMYAAW
https://www.amazon.com/gp/product/B003JMYU20
not available B003JMYU20
https://www.amazon.com/gp/product/B003JMYYQW
not available B003JMYYQW
https://www.amazon.com/gp/product/B003JN2NVO
adding to dictionary B003JN2NVO
https://www.amazon.com/gp/product/B003JN5ZEG
adding to dictionary B003JN5ZEG
https://www.amazon.com/gp/product/B003JN690U
adding to dictionary B003JN690U
https://www.amaz

adding to dictionary B003NZ1DM8
https://www.amazon.com/gp/product/B003O1INFQ
not available B003O1INFQ
https://www.amazon.com/gp/product/B003O3C06M
not available B003O3C06M
https://www.amazon.com/gp/product/B003O4BWV0
not available B003O4BWV0
https://www.amazon.com/gp/product/B003O4DNZS
not available B003O4DNZS
https://www.amazon.com/gp/product/B003O4H2L4
not available B003O4H2L4
https://www.amazon.com/gp/product/B003O52BHS
not available B003O52BHS
https://www.amazon.com/gp/product/B003O5NGJK
not available B003O5NGJK
https://www.amazon.com/gp/product/B003OBE530
not available B003OBE530
https://www.amazon.com/gp/product/B003ODETES
not available B003ODETES
https://www.amazon.com/gp/product/B003OELYFO
not available B003OELYFO
https://www.amazon.com/gp/product/B003OFHX7Q
adding to dictionary B003OFHX7Q
https://www.amazon.com/gp/product/B003OGQYC0
not available B003OGQYC0
https://www.amazon.com/gp/product/B003OGSX4M
not available B003OGSX4M
https://www.amazon.com/gp/product/B003OGT4KE
not av

not available B003SSXRME
https://www.amazon.com/gp/product/B003SSXV5C
adding to dictionary B003SSXV5C
https://www.amazon.com/gp/product/B003STPN4S
not available B003STPN4S
https://www.amazon.com/gp/product/B003SYYEG6
not available B003SYYEG6
https://www.amazon.com/gp/product/B003SZ23RC
not available B003SZ23RC
https://www.amazon.com/gp/product/B003SZEQ2W
not available B003SZEQ2W
https://www.amazon.com/gp/product/B003SZQ8SC
not available B003SZQ8SC
https://www.amazon.com/gp/product/B003T18TE6
adding to dictionary B003T18TE6
https://www.amazon.com/gp/product/B003T1G0XI
not available B003T1G0XI
https://www.amazon.com/gp/product/B003T1HFP0
not available B003T1HFP0
https://www.amazon.com/gp/product/B003T2V3FW
not available B003T2V3FW
https://www.amazon.com/gp/product/B003T4B468
not available B003T4B468
https://www.amazon.com/gp/product/B003T75D9E
not available B003T75D9E
https://www.amazon.com/gp/product/B003TA799S
not available B003TA799S
https://www.amazon.com/gp/product/B003TDZFVE
not av

not available B003VKRO6E
https://www.amazon.com/gp/product/B003VL77SI
not available B003VL77SI
https://www.amazon.com/gp/product/B003VMAL5I
not available B003VMAL5I
https://www.amazon.com/gp/product/B003VMDJW0
not available B003VMDJW0
https://www.amazon.com/gp/product/B003VMEUJ6
not available B003VMEUJ6
https://www.amazon.com/gp/product/B003VMF5S6
not available B003VMF5S6
https://www.amazon.com/gp/product/B003VMFITM
not available B003VMFITM
https://www.amazon.com/gp/product/B003VMG3RI
not available B003VMG3RI
https://www.amazon.com/gp/product/B003VMJ01Y
not available B003VMJ01Y
https://www.amazon.com/gp/product/B003VMM74Q
not available B003VMM74Q
https://www.amazon.com/gp/product/B003VMMMUK
not available B003VMMMUK
https://www.amazon.com/gp/product/B003VMOGSQ
not available B003VMOGSQ
https://www.amazon.com/gp/product/B003VMOZE6
not available B003VMOZE6
https://www.amazon.com/gp/product/B003VMS0Z6
not available B003VMS0Z6
https://www.amazon.com/gp/product/B003VMSDY4
not available B003VM

...

not available B004NMXMZ6
https://www.amazon.com/gp/product/B004NMYM96
not available B004NMYM96
https://www.amazon.com/gp/product/B004NMYMJG
not available B004NMYMJG
https://www.amazon.com/gp/product/B004NN0FIW
not available B004NN0FIW
https://www.amazon.com/gp/product/B004NN6JZA
adding to dictionary B004NN6JZA
https://www.amazon.com/gp/product/B004NU1NZE
not available B004NU1NZE
https://www.amazon.com/gp/product/B004NX17J8
adding to dictionary B004NX17J8
https://www.amazon.com/gp/product/B004NYSL9Q
not available B004NYSL9Q
https://www.amazon.com/gp/product/B004NYTR8U
not available B004NYTR8U
https://www.amazon.com/gp/product/B004NYWR7S
not available B004NYWR7S
https://www.amazon.com/gp/product/B004NYZ5AE
not available B004NYZ5AE
https://www.amazon.com/gp/product/B004NZ3D66
not available B004NZ3D66
https://www.amazon.com/gp/product/B004NZ8JLA
not available B004NZ8JLA
https://www.amazon.com/gp/product/B004NZDY42
adding to dictionary B004NZDY42
https://www.amazon.com/gp/product/B004O17UJ0

adding to dictionary B004S3M2V0
https://www.amazon.com/gp/product/B004S3PW76
not available B004S3PW76
https://www.amazon.com/gp/product/B004S6BXLW
not available B004S6BXLW
https://www.amazon.com/gp/product/B004S7X4OU
not available B004S7X4OU
https://www.amazon.com/gp/product/B004S9SEWK
not available B004S9SEWK
https://www.amazon.com/gp/product/B004S9W2SC
adding to dictionary B004S9W2SC
https://www.amazon.com/gp/product/B004SGZ610
adding to dictionary B004SGZ610
https://www.amazon.com/gp/product/B004SL0V42
not available B004SL0V42
https://www.amazon.com/gp/product/B004SODRDG
adding to dictionary B004SODRDG
https://www.amazon.com/gp/product/B004SP12LE
adding to dictionary B004SP12LE
https://www.amazon.com/gp/product/B004SPDEWE
adding to dictionary B004SPDEWE
https://www.amazon.com/gp/product/B004SPIJ0G
adding to dictionary B004SPIJ0G
https://www.amazon.com/gp/product/B004SPLCJQ
adding to dictionary B004SPLCJQ
https://www.amazon.com/gp/product/B004SPY1SU
not available B004SPY1SU
https://w

not available B004VXYHBU
https://www.amazon.com/gp/product/B004VYGQCC
not available B004VYGQCC
https://www.amazon.com/gp/product/B004VYHMU2
not available B004VYHMU2
https://www.amazon.com/gp/product/B004VYN4LS
not available B004VYN4LS
https://www.amazon.com/gp/product/B004W0OE0G
not available B004W0OE0G
https://www.amazon.com/gp/product/B004W17O7A
not available B004W17O7A
https://www.amazon.com/gp/product/B004W1F3SM
not available B004W1F3SM
https://www.amazon.com/gp/product/B004W1GRKK
not available B004W1GRKK
https://www.amazon.com/gp/product/B004W1SJDI
not available B004W1SJDI
https://www.amazon.com/gp/product/B004W24LQ6
adding to dictionary B004W24LQ6
https://www.amazon.com/gp/product/B004W2ORIS
not available B004W2ORIS
https://www.amazon.com/gp/product/B004W3GKU0
not available B004W3GKU0
https://www.amazon.com/gp/product/B004W3OR14
adding to dictionary B004W3OR14
https://www.amazon.com/gp/product/B004W4P7AI
not available B004W4P7AI
https://www.amazon.com/gp/product/B004W55086
not av

adding to dictionary B004YBW5T0
https://www.amazon.com/gp/product/B004YF5M0U
not available B004YF5M0U
https://www.amazon.com/gp/product/B004YGDPI0
not available B004YGDPI0
https://www.amazon.com/gp/product/B004YGJXC2
not available B004YGJXC2
https://www.amazon.com/gp/product/B004YJ9G6C
not available B004YJ9G6C
https://www.amazon.com/gp/product/B004YJNK3W
not available B004YJNK3W
https://www.amazon.com/gp/product/B004YL64RE
not available B004YL64RE
https://www.amazon.com/gp/product/B004YM9SVM
not available B004YM9SVM
https://www.amazon.com/gp/product/B004YNJZUA
adding to dictionary B004YNJZUA
https://www.amazon.com/gp/product/B004YRTZG0
not available B004YRTZG0
https://www.amazon.com/gp/product/B004YRUDIY
adding to dictionary B004YRUDIY
https://www.amazon.com/gp/product/B004YRUYE2
adding to dictionary B004YRUYE2
https://www.amazon.com/gp/product/B004YRVCGQ
adding to dictionary B004YRVCGQ
https://www.amazon.com/gp/product/B004YUIR9I
not available B004YUIR9I
https://www.amazon.com/gp/prod

not available B0051MQ3M6
https://www.amazon.com/gp/product/B0051MQ3TO
not available B0051MQ3TO
https://www.amazon.com/gp/product/B0051O0BG8
adding to dictionary B0051O0BG8
https://www.amazon.com/gp/product/B0051OKLUO
not available B0051OKLUO
https://www.amazon.com/gp/product/B0051OKX98
not available B0051OKX98
https://www.amazon.com/gp/product/B0051PLB6Q
not available B0051PLB6Q
https://www.amazon.com/gp/product/B0051PUWI4
adding to dictionary B0051PUWI4
https://www.amazon.com/gp/product/B0051SDC2Y
adding to dictionary B0051SDC2Y
https://www.amazon.com/gp/product/B0051SF376
not available B0051SF376
https://www.amazon.com/gp/product/B0051SX1YS
adding to dictionary B0051SX1YS
https://www.amazon.com/gp/product/B0051TSKTS
not available B0051TSKTS
https://www.amazon.com/gp/product/B0051UKI4W
not available B0051UKI4W
https://www.amazon.com/gp/product/B0051UTN3E
adding to dictionary B0051UTN3E
https://www.amazon.com/gp/product/B0051V1EQW
adding to dictionary B0051V1EQW
https://www.amazon.com/

In [40]:
print(len(list_of_products_8000))
print(len(list_of_unavailable_products_8000))

654
1346


In [41]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_8000 = pd.DataFrame(list_of_products_8000)

list_of_products_8000.to_csv('list_of_products_8000.csv')

In [42]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_8000 = pd.DataFrame(list_of_unavailable_products_8000)

list_of_unavailable_products_8000.to_csv('list_of_unavailable_products_8000.csv')

# Scraping Products 8000-10000

In [43]:

list_of_products_10000 = []
list_of_unavailable_products_10000 =[]

#Instantiating the web driver(product 8000-8500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 8000-8500 products in the asin list
for i in unique_asin[8000:8500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 8500-9000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 8500-9000 products in the asin list
for i in unique_asin[8500:9000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 7000-75000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 9000-9500 products in the asin list
for i in unique_asin[9000:9500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 9500-10000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 9500-10000 products in the asin list
for i in unique_asin[9500:10000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_10000.append(product_info)
        

    except:
        list_of_unavailable_products_10000.append(i)
        print('not available',i)


https://www.amazon.com/gp/product/B0054GLD1U
adding to dictionary B0054GLD1U
https://www.amazon.com/gp/product/B0054J38CE
not available B0054J38CE
https://www.amazon.com/gp/product/B0054JDN8S
not available B0054JDN8S
https://www.amazon.com/gp/product/B0054MS97A
not available B0054MS97A
https://www.amazon.com/gp/product/B0054MSBZA
not available B0054MSBZA
https://www.amazon.com/gp/product/B0054MSD62
not available B0054MSD62
https://www.amazon.com/gp/product/B0054QHA26
adding to dictionary B0054QHA26
https://www.amazon.com/gp/product/B0054QJCQI
not available B0054QJCQI
https://www.amazon.com/gp/product/B0054Y0FAW
not available B0054Y0FAW
https://www.amazon.com/gp/product/B0054Y0PMK
not available B0054Y0PMK
https://www.amazon.com/gp/product/B0054YMOFG
not available B0054YMOFG
https://www.amazon.com/gp/product/B00550JBQ4
not available B00550JBQ4
https://www.amazon.com/gp/product/B00557MWPE
adding to dictionary B00557MWPE
https://www.amazon.com/gp/product/B00557OUEU
not available B00557OUEU

not available B005AW82Z8
https://www.amazon.com/gp/product/B005AYHHNO
not available B005AYHHNO
https://www.amazon.com/gp/product/B005AYJ6EW
not available B005AYJ6EW
https://www.amazon.com/gp/product/B005AZFREE
adding to dictionary B005AZFREE
https://www.amazon.com/gp/product/B005AZSE44
adding to dictionary B005AZSE44
https://www.amazon.com/gp/product/B005B3OBM4
not available B005B3OBM4
https://www.amazon.com/gp/product/B005B9NWRS
not available B005B9NWRS
https://www.amazon.com/gp/product/B005BDO4TY
not available B005BDO4TY
https://www.amazon.com/gp/product/B005BEF5EQ
adding to dictionary B005BEF5EQ
https://www.amazon.com/gp/product/B005BF1M10
not available B005BF1M10
https://www.amazon.com/gp/product/B005BG02BU
not available B005BG02BU
https://www.amazon.com/gp/product/B005BH9LKC
not available B005BH9LKC
https://www.amazon.com/gp/product/B005BIO4Q2
not available B005BIO4Q2
https://www.amazon.com/gp/product/B005BQ7J5W
not available B005BQ7J5W
https://www.amazon.com/gp/product/B005BQ7K8S

not available B005FKTDLW
https://www.amazon.com/gp/product/B005FMTAC2
not available B005FMTAC2
https://www.amazon.com/gp/product/B005FNVINA
not available B005FNVINA
https://www.amazon.com/gp/product/B005FO1S0W
not available B005FO1S0W
https://www.amazon.com/gp/product/B005FOPQT6
not available B005FOPQT6
https://www.amazon.com/gp/product/B005FPN0I4
not available B005FPN0I4
https://www.amazon.com/gp/product/B005FPWORC
not available B005FPWORC
https://www.amazon.com/gp/product/B005FRGJ6M
not available B005FRGJ6M
https://www.amazon.com/gp/product/B005FU4DTE
not available B005FU4DTE
https://www.amazon.com/gp/product/B005FU9SDU
adding to dictionary B005FU9SDU
https://www.amazon.com/gp/product/B005FVSH0O
adding to dictionary B005FVSH0O
https://www.amazon.com/gp/product/B005FW9JR8
not available B005FW9JR8
https://www.amazon.com/gp/product/B005FYAQRI
not available B005FYAQRI
https://www.amazon.com/gp/product/B005FYJB92
adding to dictionary B005FYJB92
https://www.amazon.com/gp/product/B005FYTGYC

adding to dictionary B005J4Y0O4
https://www.amazon.com/gp/product/B005J5SGS4
not available B005J5SGS4
https://www.amazon.com/gp/product/B005J5T5VG
not available B005J5T5VG
https://www.amazon.com/gp/product/B005J5TIYK
not available B005J5TIYK
https://www.amazon.com/gp/product/B005J5TO0S
not available B005J5TO0S
https://www.amazon.com/gp/product/B005JD4NZQ
not available B005JD4NZQ
https://www.amazon.com/gp/product/B005JDPJ3G
not available B005JDPJ3G
https://www.amazon.com/gp/product/B005JFPGX2
not available B005JFPGX2
https://www.amazon.com/gp/product/B005JJ4GX4
adding to dictionary B005JJ4GX4
https://www.amazon.com/gp/product/B005JLLG8U
not available B005JLLG8U
https://www.amazon.com/gp/product/B005JNK9XQ
not available B005JNK9XQ
https://www.amazon.com/gp/product/B005JPAZHE
not available B005JPAZHE
https://www.amazon.com/gp/product/B005JPBFBY
not available B005JPBFBY
https://www.amazon.com/gp/product/B005JQQKXQ
not available B005JQQKXQ
https://www.amazon.com/gp/product/B005JRGH0G
not av

adding to dictionary B005OFAARY
https://www.amazon.com/gp/product/B005OHQUTO
adding to dictionary B005OHQUTO
https://www.amazon.com/gp/product/B005OKCM2U
not available B005OKCM2U
https://www.amazon.com/gp/product/B005OKPZR4
not available B005OKPZR4
https://www.amazon.com/gp/product/B005OKQ1A4
adding to dictionary B005OKQ1A4
https://www.amazon.com/gp/product/B005OKUXEO
adding to dictionary B005OKUXEO
https://www.amazon.com/gp/product/B005OKYG9M
not available B005OKYG9M
https://www.amazon.com/gp/product/B005OMZW48
adding to dictionary B005OMZW48
https://www.amazon.com/gp/product/B005OS52HE
not available B005OS52HE
https://www.amazon.com/gp/product/B005OS8NZC
not available B005OS8NZC
https://www.amazon.com/gp/product/B005OSQGN8
adding to dictionary B005OSQGN8
https://www.amazon.com/gp/product/B005OTEH3S
adding to dictionary B005OTEH3S
https://www.amazon.com/gp/product/B005OU2Y1E
adding to dictionary B005OU2Y1E
https://www.amazon.com/gp/product/B005OU4J6C
adding to dictionary B005OU4J6C
ht

...

not available B007LA6CQS
https://www.amazon.com/gp/product/B007LANY2S
not available B007LANY2S
https://www.amazon.com/gp/product/B007LSASW4
not available B007LSASW4
https://www.amazon.com/gp/product/B007M0S8KA
not available B007M0S8KA
https://www.amazon.com/gp/product/B007M0STLI
not available B007M0STLI
https://www.amazon.com/gp/product/B007M4D49Q
not available B007M4D49Q
https://www.amazon.com/gp/product/B007M6CTW2
not available B007M6CTW2
https://www.amazon.com/gp/product/B007M6EALK
adding to dictionary B007M6EALK
https://www.amazon.com/gp/product/B007M819DU
adding to dictionary B007M819DU
https://www.amazon.com/gp/product/B007M81A8O
not available B007M81A8O
https://www.amazon.com/gp/product/B007M81AAC
not available B007M81AAC
https://www.amazon.com/gp/product/B007M81B4M
adding to dictionary B007M81B4M
https://www.amazon.com/gp/product/B007MBE786
not available B007MBE786
https://www.amazon.com/gp/product/B007MF5UOM
not available B007MF5UOM
https://www.amazon.com/gp/product/B007MF8846

not available B007S5A2K8
https://www.amazon.com/gp/product/B007S5BB8A
not available B007S5BB8A
https://www.amazon.com/gp/product/B007S69QAY
adding to dictionary B007S69QAY
https://www.amazon.com/gp/product/B007SA3CD2
not available B007SA3CD2
https://www.amazon.com/gp/product/B007SD28UM
adding to dictionary B007SD28UM
https://www.amazon.com/gp/product/B007SEEAEI
not available B007SEEAEI
https://www.amazon.com/gp/product/B007SG2QT2
not available B007SG2QT2
https://www.amazon.com/gp/product/B007SIF4BC
adding to dictionary B007SIF4BC
https://www.amazon.com/gp/product/B007SIG5HY
not available B007SIG5HY
https://www.amazon.com/gp/product/B007SLCOXU
adding to dictionary B007SLCOXU
https://www.amazon.com/gp/product/B007SXVCWC
adding to dictionary B007SXVCWC
https://www.amazon.com/gp/product/B007SZCG0C
adding to dictionary B007SZCG0C
https://www.amazon.com/gp/product/B007TA89ZM
not available B007TA89ZM
https://www.amazon.com/gp/product/B007TAQZ8A
adding to dictionary B007TAQZ8A
https://www.amaz

not available B00812ZWOS
https://www.amazon.com/gp/product/B008134WZ2
not available B008134WZ2
https://www.amazon.com/gp/product/B008148R8E
not available B008148R8E
https://www.amazon.com/gp/product/B00819JRNS
adding to dictionary B00819JRNS
https://www.amazon.com/gp/product/B0081A2G5I
not available B0081A2G5I
https://www.amazon.com/gp/product/B0081CWUD4
not available B0081CWUD4
https://www.amazon.com/gp/product/B0081CWUE8
adding to dictionary B0081CWUE8
https://www.amazon.com/gp/product/B0081E6THU
not available B0081E6THU
https://www.amazon.com/gp/product/B0081E6TKC
not available B0081E6TKC
https://www.amazon.com/gp/product/B0081E6X1W
not available B0081E6X1W
https://www.amazon.com/gp/product/B0081E6X2Q
adding to dictionary B0081E6X2Q
https://www.amazon.com/gp/product/B0081E6X3U
not available B0081E6X3U
https://www.amazon.com/gp/product/B0081E6YRK
adding to dictionary B0081E6YRK
https://www.amazon.com/gp/product/B0081E6YX4
adding to dictionary B0081E6YX4
https://www.amazon.com/gp/prod

not available B0086LC0L2
https://www.amazon.com/gp/product/B0086LC0TO
not available B0086LC0TO
https://www.amazon.com/gp/product/B0086LC0WG
not available B0086LC0WG
https://www.amazon.com/gp/product/B0086LC1U2
not available B0086LC1U2
https://www.amazon.com/gp/product/B0086LC3KK
not available B0086LC3KK
https://www.amazon.com/gp/product/B0086LC47M
not available B0086LC47M
https://www.amazon.com/gp/product/B0086LC49K
not available B0086LC49K
https://www.amazon.com/gp/product/B0086XL9DK
adding to dictionary B0086XL9DK
https://www.amazon.com/gp/product/B0086XX0AK
not available B0086XX0AK
https://www.amazon.com/gp/product/B0086YV8BM
not available B0086YV8BM
https://www.amazon.com/gp/product/B00870XLDS
adding to dictionary B00870XLDS
https://www.amazon.com/gp/product/B00871KLEE
not available B00871KLEE
https://www.amazon.com/gp/product/B00871KLRQ
not available B00871KLRQ
https://www.amazon.com/gp/product/B00871KM98
not available B00871KM98
https://www.amazon.com/gp/product/B00871KMCU
not av

adding to dictionary B008CEDX8M
https://www.amazon.com/gp/product/B008CEDY5O
adding to dictionary B008CEDY5O
https://www.amazon.com/gp/product/B008COP6NM
not available B008COP6NM
https://www.amazon.com/gp/product/B008CT04I4
not available B008CT04I4
https://www.amazon.com/gp/product/B008CT12G2
adding to dictionary B008CT12G2
https://www.amazon.com/gp/product/B008CZBZMM
not available B008CZBZMM
https://www.amazon.com/gp/product/B008CZBZY0
adding to dictionary B008CZBZY0
https://www.amazon.com/gp/product/B008CZC1IE
not available B008CZC1IE
https://www.amazon.com/gp/product/B008D3JPKM
not available B008D3JPKM
https://www.amazon.com/gp/product/B008D4DFE8
not available B008D4DFE8
https://www.amazon.com/gp/product/B008D5D9NY
adding to dictionary B008D5D9NY
https://www.amazon.com/gp/product/B008D5HAHU
adding to dictionary B008D5HAHU
https://www.amazon.com/gp/product/B008D5I61Y
not available B008D5I61Y
https://www.amazon.com/gp/product/B008D5ZR5M
not available B008D5ZR5M
https://www.amazon.com/

In [44]:
print(len(list_of_products_10000))
print(len(list_of_unavailable_products_10000))

567
1433


In [45]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_10000 = pd.DataFrame(list_of_products_10000)

list_of_products_10000.to_csv('list_of_products_10000.csv')

In [46]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_10000 = pd.DataFrame(list_of_unavailable_products_10000)

list_of_unavailable_products_10000.to_csv('list_of_unavailable_products_10000.csv')

# # Scraping products 10000-12101

In [49]:
list_of_products_12000 = []
list_of_unavailable_products_12000 =[]


#Instantiating the web driver(product 10000-10500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in unique_asin[10000:10500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 10500-11000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 10500-11000 products in the asin list
for i in unique_asin[10500:11000]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 11000-115000)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 11000-11500 products in the asin list
for i in unique_asin[11000:11500]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)
        
#Instantiating the web driver(product 11500-12101)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through 11500-12101 products in the asin list
for i in unique_asin[11500:12101]:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_products_12000.append(product_info)
        

    except:
        list_of_unavailable_products_12000.append(i)
        print('not available',i)


https://www.amazon.com/gp/product/B008GFVOOM
not available B008GFVOOM
https://www.amazon.com/gp/product/B008GO04PS
not available B008GO04PS
https://www.amazon.com/gp/product/B008GOR6O0
not available B008GOR6O0
https://www.amazon.com/gp/product/B008GRDTPW
not available B008GRDTPW
https://www.amazon.com/gp/product/B008GSY7UM
adding to dictionary B008GSY7UM
https://www.amazon.com/gp/product/B008H1IDKI
not available B008H1IDKI
https://www.amazon.com/gp/product/B008H2UTFE
not available B008H2UTFE
https://www.amazon.com/gp/product/B008H3YYA4
not available B008H3YYA4
https://www.amazon.com/gp/product/B008H42554
not available B008H42554
https://www.amazon.com/gp/product/B008H5GGGM
not available B008H5GGGM
https://www.amazon.com/gp/product/B008H7RWH2
not available B008H7RWH2
https://www.amazon.com/gp/product/B008HBMQTM
adding to dictionary B008HBMQTM
https://www.amazon.com/gp/product/B008HBMRIC
adding to dictionary B008HBMRIC
https://www.amazon.com/gp/product/B008HBMRRS
not available B008HBMRRS

not available B008QMPTSM
https://www.amazon.com/gp/product/B008QMWPKM
not available B008QMWPKM
https://www.amazon.com/gp/product/B008QMWUMK
not available B008QMWUMK
https://www.amazon.com/gp/product/B008QMWVP6
adding to dictionary B008QMWVP6
https://www.amazon.com/gp/product/B008QWW9GM
not available B008QWW9GM
https://www.amazon.com/gp/product/B008QZUHZE
not available B008QZUHZE
https://www.amazon.com/gp/product/B008R1JSKW
adding to dictionary B008R1JSKW
https://www.amazon.com/gp/product/B008R56PAY
not available B008R56PAY
https://www.amazon.com/gp/product/B008R56PVI
not available B008R56PVI
https://www.amazon.com/gp/product/B008RAMOYA
adding to dictionary B008RAMOYA
https://www.amazon.com/gp/product/B008RG9MKS
not available B008RG9MKS
https://www.amazon.com/gp/product/B008RIJVTI
not available B008RIJVTI
https://www.amazon.com/gp/product/B008RLQLWA
not available B008RLQLWA
https://www.amazon.com/gp/product/B008RM6V4M
not available B008RM6V4M
https://www.amazon.com/gp/product/B008RRV7R8

not available B008YKNHN0
https://www.amazon.com/gp/product/B008YPNSNE
not available B008YPNSNE
https://www.amazon.com/gp/product/B008YQ0R9G
not available B008YQ0R9G
https://www.amazon.com/gp/product/B008YRZA24
not available B008YRZA24
https://www.amazon.com/gp/product/B008YS8F7A
not available B008YS8F7A
https://www.amazon.com/gp/product/B008YSQLU8
not available B008YSQLU8
https://www.amazon.com/gp/product/B008Z016X2
adding to dictionary B008Z016X2
https://www.amazon.com/gp/product/B008ZFHV5Y
not available B008ZFHV5Y
https://www.amazon.com/gp/product/B008ZHP9W4
not available B008ZHP9W4
https://www.amazon.com/gp/product/B008ZIFQGC
not available B008ZIFQGC
https://www.amazon.com/gp/product/B00902KYPU
not available B00902KYPU
https://www.amazon.com/gp/product/B009062W9W
not available B009062W9W
https://www.amazon.com/gp/product/B0090ELFQK
not available B0090ELFQK
https://www.amazon.com/gp/product/B0090ELFRO
not available B0090ELFRO
https://www.amazon.com/gp/product/B0090PHG7Q
adding to dic

adding to dictionary B009AVFJ92
https://www.amazon.com/gp/product/B009B7E450
adding to dictionary B009B7E450
https://www.amazon.com/gp/product/B009C650MQ
adding to dictionary B009C650MQ
https://www.amazon.com/gp/product/B009C7IRZW
not available B009C7IRZW
https://www.amazon.com/gp/product/B009CA7T4O
not available B009CA7T4O
https://www.amazon.com/gp/product/B009CA7T5I
not available B009CA7T5I
https://www.amazon.com/gp/product/B009CNA3ZI
adding to dictionary B009CNA3ZI
https://www.amazon.com/gp/product/B009CRUDRW
not available B009CRUDRW
https://www.amazon.com/gp/product/B009CS493U
not available B009CS493U
https://www.amazon.com/gp/product/B009CSJB56
not available B009CSJB56
https://www.amazon.com/gp/product/B009CUYUVY
not available B009CUYUVY
https://www.amazon.com/gp/product/B009CZR3Y0
adding to dictionary B009CZR3Y0
https://www.amazon.com/gp/product/B009D7V4WO
not available B009D7V4WO
https://www.amazon.com/gp/product/B009D9O6YA
not available B009D9O6YA
https://www.amazon.com/gp/prod

adding to dictionary B009O5BC22
https://www.amazon.com/gp/product/B009OWLX8I
not available B009OWLX8I
https://www.amazon.com/gp/product/B009OY2REK
not available B009OY2REK
https://www.amazon.com/gp/product/B009OY8GK4
not available B009OY8GK4
https://www.amazon.com/gp/product/B009P3CCYA
not available B009P3CCYA
https://www.amazon.com/gp/product/B009P3DJZG
not available B009P3DJZG
https://www.amazon.com/gp/product/B009P4ZJXK
adding to dictionary B009P4ZJXK
https://www.amazon.com/gp/product/B009P98O6E
adding to dictionary B009P98O6E
https://www.amazon.com/gp/product/B009PVHU1W
not available B009PVHU1W
https://www.amazon.com/gp/product/B009PZF7WM
not available B009PZF7WM
https://www.amazon.com/gp/product/B009PZGYJM
not available B009PZGYJM
https://www.amazon.com/gp/product/B009PZVOF6
not available B009PZVOF6
https://www.amazon.com/gp/product/B009Q1YPNW
not available B009Q1YPNW
https://www.amazon.com/gp/product/B009Q1YPV4
not available B009Q1YPV4
https://www.amazon.com/gp/product/B009Q1YPVY

...

adding to dictionary B00FH692PQ
https://www.amazon.com/gp/product/B00FI9BOE4
not available B00FI9BOE4
https://www.amazon.com/gp/product/B00FIUQ498
adding to dictionary B00FIUQ498
https://www.amazon.com/gp/product/B00FJE4K94
not available B00FJE4K94
https://www.amazon.com/gp/product/B00FJF5BXM
not available B00FJF5BXM
https://www.amazon.com/gp/product/B00FKSWN8E
not available B00FKSWN8E
https://www.amazon.com/gp/product/B00FKSWNPM
not available B00FKSWNPM
https://www.amazon.com/gp/product/B00FLKP5UO
not available B00FLKP5UO
https://www.amazon.com/gp/product/B00FLRHYCE
adding to dictionary B00FLRHYCE
https://www.amazon.com/gp/product/B00FOGWSOG
adding to dictionary B00FOGWSOG
https://www.amazon.com/gp/product/B00FP6OMRG
not available B00FP6OMRG
https://www.amazon.com/gp/product/B00FPROWWU
adding to dictionary B00FPROWWU
https://www.amazon.com/gp/product/B00FQGNHBM
not available B00FQGNHBM
https://www.amazon.com/gp/product/B00FRDPMQC
not available B00FRDPMQC
https://www.amazon.com/gp/prod

not available B00GMCP1R2
https://www.amazon.com/gp/product/B00GMDIKV0
not available B00GMDIKV0
https://www.amazon.com/gp/product/B00GMIXWTU
not available B00GMIXWTU
https://www.amazon.com/gp/product/B00GMMJKUQ
adding to dictionary B00GMMJKUQ
https://www.amazon.com/gp/product/B00GMQDN64
adding to dictionary B00GMQDN64
https://www.amazon.com/gp/product/B00GMRJWPY
not available B00GMRJWPY
https://www.amazon.com/gp/product/B00GMWMMOC
not available B00GMWMMOC
https://www.amazon.com/gp/product/B00GMWMMOW
not available B00GMWMMOW
https://www.amazon.com/gp/product/B00GMZUS2M
adding to dictionary B00GMZUS2M
https://www.amazon.com/gp/product/B00GN5L14A
not available B00GN5L14A
https://www.amazon.com/gp/product/B00GNA9K3E
adding to dictionary B00GNA9K3E
https://www.amazon.com/gp/product/B00GNA9KKM
adding to dictionary B00GNA9KKM
https://www.amazon.com/gp/product/B00GNA9W4G
not available B00GNA9W4G
https://www.amazon.com/gp/product/B00GNA9WG4
adding to dictionary B00GNA9WG4
https://www.amazon.com/

not available B00H8Z2LNY
https://www.amazon.com/gp/product/B00H8ZLVTE
adding to dictionary B00H8ZLVTE
https://www.amazon.com/gp/product/B00H927WRG
not available B00H927WRG
https://www.amazon.com/gp/product/B00H92CA8M
not available B00H92CA8M
https://www.amazon.com/gp/product/B00H93NJLS
not available B00H93NJLS
https://www.amazon.com/gp/product/B00H941PVS
not available B00H941PVS
https://www.amazon.com/gp/product/B00H997F6M
adding to dictionary B00H997F6M
https://www.amazon.com/gp/product/B00H9BLUES
not available B00H9BLUES
https://www.amazon.com/gp/product/B00HADEHUO
not available B00HADEHUO
https://www.amazon.com/gp/product/B00HAPQT7Q
not available B00HAPQT7Q
https://www.amazon.com/gp/product/B00HAQAREQ
adding to dictionary B00HAQAREQ
https://www.amazon.com/gp/product/B00HAQH3NY
adding to dictionary B00HAQH3NY
https://www.amazon.com/gp/product/B00HAT8EKW
adding to dictionary B00HAT8EKW
https://www.amazon.com/gp/product/B00HAZ4MBQ
not available B00HAZ4MBQ
https://www.amazon.com/gp/prod

adding to dictionary B00IFVHJBO
https://www.amazon.com/gp/product/B00IFVHJCI
adding to dictionary B00IFVHJCI
https://www.amazon.com/gp/product/B00IFVHJDM
adding to dictionary B00IFVHJDM
https://www.amazon.com/gp/product/B00IFVHJE6
adding to dictionary B00IFVHJE6
https://www.amazon.com/gp/product/B00IG8ZS1E
adding to dictionary B00IG8ZS1E
https://www.amazon.com/gp/product/B00IG9L8QM
adding to dictionary B00IG9L8QM
https://www.amazon.com/gp/product/B00IH49VBU
not available B00IH49VBU
https://www.amazon.com/gp/product/B00IIB82FS
adding to dictionary B00IIB82FS
https://www.amazon.com/gp/product/B00IIBB7A0
adding to dictionary B00IIBB7A0
https://www.amazon.com/gp/product/B00IIFOEM4
not available B00IIFOEM4
https://www.amazon.com/gp/product/B00IJYHRAK
not available B00IJYHRAK
https://www.amazon.com/gp/product/B00IJZJ9K0
not available B00IJZJ9K0
https://www.amazon.com/gp/product/B00IJZMXT4
not available B00IJZMXT4
https://www.amazon.com/gp/product/B00IK5DICE
not available B00IK5DICE
https://w

not available B00KHH2VOY
https://www.amazon.com/gp/product/B00KI5UL8I
not available B00KI5UL8I
https://www.amazon.com/gp/product/B00KIEGBHE
not available B00KIEGBHE
https://www.amazon.com/gp/product/B00KK1V28I
not available B00KK1V28I
https://www.amazon.com/gp/product/B00KK62714
not available B00KK62714
https://www.amazon.com/gp/product/B00KKKW03U
not available B00KKKW03U
https://www.amazon.com/gp/product/B00KLA4INE
not available B00KLA4INE
https://www.amazon.com/gp/product/B00KLJDYL2
not available B00KLJDYL2
https://www.amazon.com/gp/product/B00KLZO2JE
not available B00KLZO2JE
https://www.amazon.com/gp/product/B00KNIL36E
not available B00KNIL36E
https://www.amazon.com/gp/product/B00KNJOLSA
not available B00KNJOLSA
https://www.amazon.com/gp/product/B00KPKIV8I
not available B00KPKIV8I
https://www.amazon.com/gp/product/B00KQ4PEBU
not available B00KQ4PEBU
https://www.amazon.com/gp/product/B00KQBR9FM
not available B00KQBR9FM
https://www.amazon.com/gp/product/B00KQVTX06
not available B00KQV

In [50]:
print(len(list_of_products_12000))
print(len(list_of_unavailable_products_12000))

509
1592


In [51]:
#Change the list to pandas dataframe to write onto a csv file
list_of_products_12000 = pd.DataFrame(list_of_products_12000)

list_of_products_12000.to_csv('list_of_products_12000.csv')

In [52]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_12000 = pd.DataFrame(list_of_unavailable_products_12000)

list_of_unavailable_products_12000.to_csv('list_of_unavailable_products_12000.csv')

Scraping the unavailable columns to check for anti robot block

In [59]:
type(list_of_unavailable_products_2000)

list

In [58]:
list_of_unavailable_products_2000=list_of_unavailable_products_2000.values.tolist()

In [62]:
list_of_unavailable_products_2000[:10]

[['7806397051'],
 ['9759091062'],
 ['9790790961'],
 ['9790794231'],
 ['B00004U9UY'],
 ['B00004U9V2'],
 ['B000050B6U'],
 ['B000052WY7'],
 ['B000052WYL'],
 ['B000052XZP']]

In [75]:
list_of_unavailable_products_2000=oneDArray(list_of_unavailable_products_2000)

In [76]:
list_of_unavailable_products_2000[:10]

['7806397051',
 '9759091062',
 '9790790961',
 '9790794231',
 'B00004U9UY',
 'B00004U9V2',
 'B000050B6U',
 'B000052WY7',
 'B000052WYL',
 'B000052XZP']

In [77]:

list_of_available_products_Rerun_2000 = []
list_of_unavailable_products_Rerun_2000 =[]


#Instantiating the web driver(product 10000-10500)
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_2000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_2000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_2000.append(i)
        print('not available',i)
        


https://www.amazon.com/gp/product/7806397051
not available 7806397051
https://www.amazon.com/gp/product/9759091062
not available 9759091062
https://www.amazon.com/gp/product/9790790961
not available 9790790961
https://www.amazon.com/gp/product/9790794231
not available 9790794231
https://www.amazon.com/gp/product/B00004U9UY
not available B00004U9UY
https://www.amazon.com/gp/product/B00004U9V2
not available B00004U9V2
https://www.amazon.com/gp/product/B000050B6U
not available B000050B6U
https://www.amazon.com/gp/product/B000052WY7
not available B000052WY7
https://www.amazon.com/gp/product/B000052WYL
not available B000052WYL
https://www.amazon.com/gp/product/B000052XZP
not available B000052XZP
https://www.amazon.com/gp/product/B000052Y25
not available B000052Y25
https://www.amazon.com/gp/product/B000052Y5F
not available B000052Y5F
https://www.amazon.com/gp/product/B000052Y6Z
not available B000052Y6Z
https://www.amazon.com/gp/product/B000052YJD
not available B000052YJD
https://www.amazon.c

not available B0000CNR0J
https://www.amazon.com/gp/product/B0000Y3C3I
not available B0000Y3C3I
https://www.amazon.com/gp/product/B0000Y3COC
not available B0000Y3COC
https://www.amazon.com/gp/product/B0000Y3CSI
not available B0000Y3CSI
https://www.amazon.com/gp/product/B0000Y3D4G
not available B0000Y3D4G
https://www.amazon.com/gp/product/B0000Y3FRG
not available B0000Y3FRG
https://www.amazon.com/gp/product/B0000YUWY0
not available B0000YUWY0
https://www.amazon.com/gp/product/B0000YUX4O
not available B0000YUX4O
https://www.amazon.com/gp/product/B0000YV83O
not available B0000YV83O
https://www.amazon.com/gp/product/B0000ZLEFU
not available B0000ZLEFU
https://www.amazon.com/gp/product/B0000ZLEX2
not available B0000ZLEX2
https://www.amazon.com/gp/product/B00011JI88
not available B00011JI88
https://www.amazon.com/gp/product/B00011JTFA
not available B00011JTFA
https://www.amazon.com/gp/product/B00011QUDE
not available B00011QUDE
https://www.amazon.com/gp/product/B00011QVIS
not available B00011

not available B000225ZOC
https://www.amazon.com/gp/product/B0002260DW
not available B0002260DW
https://www.amazon.com/gp/product/B0002260L4
not available B0002260L4
https://www.amazon.com/gp/product/B0002260N2
not available B0002260N2
https://www.amazon.com/gp/product/B00022WA9U
not available B00022WA9U
https://www.amazon.com/gp/product/B000248HLI
not available B000248HLI
https://www.amazon.com/gp/product/B000260KIO
not available B000260KIO
https://www.amazon.com/gp/product/B000273RI8
not available B000273RI8
https://www.amazon.com/gp/product/B000277N7Y
not available B000277N7Y
https://www.amazon.com/gp/product/B00027C9CS
not available B00027C9CS
https://www.amazon.com/gp/product/B00027CDY2
not available B00027CDY2
https://www.amazon.com/gp/product/B00027CDYM
not available B00027CDYM
https://www.amazon.com/gp/product/B00027CGWQ
not available B00027CGWQ
https://www.amazon.com/gp/product/B00027D0DU
not available B00027D0DU
https://www.amazon.com/gp/product/B00027D98Q
not available B00027

not available B0006II74C
https://www.amazon.com/gp/product/B0006IJA5C
not available B0006IJA5C
https://www.amazon.com/gp/product/B0006IXSG4
not available B0006IXSG4
https://www.amazon.com/gp/product/B0006LNC2G
not available B0006LNC2G
https://www.amazon.com/gp/product/B0006LNKXW
not available B0006LNKXW
https://www.amazon.com/gp/product/B0006LNMIU
not available B0006LNMIU
https://www.amazon.com/gp/product/B0006M5566
not available B0006M5566
https://www.amazon.com/gp/product/B0006MSVCG
not available B0006MSVCG
https://www.amazon.com/gp/product/B0006N29AU
not available B0006N29AU
https://www.amazon.com/gp/product/B0006NY35E
not available B0006NY35E
https://www.amazon.com/gp/product/B0006NYDOU
not available B0006NYDOU
https://www.amazon.com/gp/product/B0006PJRP8
not available B0006PJRP8
https://www.amazon.com/gp/product/B0006PJSES
not available B0006PJSES
https://www.amazon.com/gp/product/B0006PJTB0
not available B0006PJTB0
https://www.amazon.com/gp/product/B0006PKGJY
not available B0006P

not available B0009OAHWI
https://www.amazon.com/gp/product/B0009OAI18
not available B0009OAI18
https://www.amazon.com/gp/product/B0009OAI2C
not available B0009OAI2C
https://www.amazon.com/gp/product/B0009OAI3G
not available B0009OAI3G
https://www.amazon.com/gp/product/B0009OAI40
not available B0009OAI40
https://www.amazon.com/gp/product/B0009OAIB8
not available B0009OAIB8
https://www.amazon.com/gp/product/B0009OAIE0
not available B0009OAIE0
https://www.amazon.com/gp/product/B0009OAIQI
not available B0009OAIQI
https://www.amazon.com/gp/product/B0009OAISG
not available B0009OAISG
https://www.amazon.com/gp/product/B0009OMNGQ
not available B0009OMNGQ
https://www.amazon.com/gp/product/B0009P4PZC
not available B0009P4PZC
https://www.amazon.com/gp/product/B0009PVV36
not available B0009PVV36
https://www.amazon.com/gp/product/B0009QJ3TE
not available B0009QJ3TE
https://www.amazon.com/gp/product/B0009QZXP2
not available B0009QZXP2
https://www.amazon.com/gp/product/B0009QZYN8
not available B0009Q

not available B000C1VT70
https://www.amazon.com/gp/product/B000C1VWAE
not available B000C1VWAE
https://www.amazon.com/gp/product/B000C1VWCM
not available B000C1VWCM
https://www.amazon.com/gp/product/B000C1VWJU
not available B000C1VWJU
https://www.amazon.com/gp/product/B000C1VWTA
not available B000C1VWTA
https://www.amazon.com/gp/product/B000C1W1DG
not available B000C1W1DG
https://www.amazon.com/gp/product/B000C1W2C6
not available B000C1W2C6
https://www.amazon.com/gp/product/B000C1W2DK
not available B000C1W2DK
https://www.amazon.com/gp/product/B000C1W4RE
not available B000C1W4RE
https://www.amazon.com/gp/product/B000C1W4T2
not available B000C1W4T2
https://www.amazon.com/gp/product/B000C1W6TK
not available B000C1W6TK
https://www.amazon.com/gp/product/B000C1Z1H4
not available B000C1Z1H4
https://www.amazon.com/gp/product/B000C1Z1HE
not available B000C1Z1HE
https://www.amazon.com/gp/product/B000C1Z1UG
not available B000C1Z1UG
https://www.amazon.com/gp/product/B000C1Z29Q
not available B000C1

not available B000F3KXV8
https://www.amazon.com/gp/product/B000F3XXZG
not available B000F3XXZG
https://www.amazon.com/gp/product/B000F3ZSVI
not available B000F3ZSVI
https://www.amazon.com/gp/product/B000F41ZEG
not available B000F41ZEG
https://www.amazon.com/gp/product/B000F4DLBG
not available B000F4DLBG
https://www.amazon.com/gp/product/B000F4GIPC
not available B000F4GIPC
https://www.amazon.com/gp/product/B000F4GIPM
not available B000F4GIPM
https://www.amazon.com/gp/product/B000F4GLQS
not available B000F4GLQS
https://www.amazon.com/gp/product/B000F4IL70
not available B000F4IL70
https://www.amazon.com/gp/product/B000F5AFYG
not available B000F5AFYG
https://www.amazon.com/gp/product/B000F63TW0
not available B000F63TW0
https://www.amazon.com/gp/product/B000F6NXJY
not available B000F6NXJY
https://www.amazon.com/gp/product/B000F7GAQQ
not available B000F7GAQQ
https://www.amazon.com/gp/product/B000F8FZAC
not available B000F8FZAC
https://www.amazon.com/gp/product/B000F8HWXU
not available B000F8

not available B000GHF3JA
https://www.amazon.com/gp/product/B000GHGWKY
not available B000GHGWKY
https://www.amazon.com/gp/product/B000GHIBQM
not available B000GHIBQM
https://www.amazon.com/gp/product/B000GHVCM2
not available B000GHVCM2
https://www.amazon.com/gp/product/B000GHWSG6
not available B000GHWSG6
https://www.amazon.com/gp/product/B000GHYSL4
not available B000GHYSL4
https://www.amazon.com/gp/product/B000GI0WAY
not available B000GI0WAY
https://www.amazon.com/gp/product/B000GI3TXQ
not available B000GI3TXQ
https://www.amazon.com/gp/product/B000GI3U4Y
not available B000GI3U4Y
https://www.amazon.com/gp/product/B000GJL22K
not available B000GJL22K
https://www.amazon.com/gp/product/B000GNUUCO
not available B000GNUUCO
https://www.amazon.com/gp/product/B000GP78S6
not available B000GP78S6
https://www.amazon.com/gp/product/B000GRENPA
not available B000GRENPA
https://www.amazon.com/gp/product/B000GREPOY
not available B000GREPOY
https://www.amazon.com/gp/product/B000GS8T0Y
not available B000GS

not available B000K5E83K
https://www.amazon.com/gp/product/B000K7E2G6
not available B000K7E2G6
https://www.amazon.com/gp/product/B000K7J1PI
not available B000K7J1PI
https://www.amazon.com/gp/product/B000K7JLGM
not available B000K7JLGM
https://www.amazon.com/gp/product/B000K9NZJE
not available B000K9NZJE
https://www.amazon.com/gp/product/B000KEOO3U
not available B000KEOO3U
https://www.amazon.com/gp/product/B000KEOQR4
not available B000KEOQR4
https://www.amazon.com/gp/product/B000KFVY46
not available B000KFVY46
https://www.amazon.com/gp/product/B000KFX10G
not available B000KFX10G
https://www.amazon.com/gp/product/B000KFXZ20
not available B000KFXZ20
https://www.amazon.com/gp/product/B000KHTDM4
not available B000KHTDM4
https://www.amazon.com/gp/product/B000KIRSAW
not available B000KIRSAW
https://www.amazon.com/gp/product/B000KJXKAS
not available B000KJXKAS
https://www.amazon.com/gp/product/B000KK53L6
not available B000KK53L6
https://www.amazon.com/gp/product/B000KM1PVG
not available B000KM

not available B000NSIGE6
https://www.amazon.com/gp/product/B000NSQI4Q
not available B000NSQI4Q
https://www.amazon.com/gp/product/B000NSTX20
adding to dictionary B000NSTX20
https://www.amazon.com/gp/product/B000NUMRXK
not available B000NUMRXK
https://www.amazon.com/gp/product/B000NUTUMQ
not available B000NUTUMQ
https://www.amazon.com/gp/product/B000NUW8W0
not available B000NUW8W0
https://www.amazon.com/gp/product/B000NVFWB8
not available B000NVFWB8
https://www.amazon.com/gp/product/B000NVIFFS
not available B000NVIFFS
https://www.amazon.com/gp/product/B000NW4LDM
not available B000NW4LDM
https://www.amazon.com/gp/product/B000NY66DS
not available B000NY66DS
https://www.amazon.com/gp/product/B000NYL3NQ
not available B000NYL3NQ
https://www.amazon.com/gp/product/B000NYL3UE
not available B000NYL3UE
https://www.amazon.com/gp/product/B000NZAIC2
not available B000NZAIC2
https://www.amazon.com/gp/product/B000NZN2V6
not available B000NZN2V6
https://www.amazon.com/gp/product/B000O03F4O
not available

In [79]:
print(len(list_of_available_products_Rerun_2000))
print(len(list_of_unavailable_products_Rerun_2000))

31
1123


In [83]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_2000 = pd.DataFrame(list_of_available_products_Rerun_2000)

list_of_available_products_Rerun_2000.to_csv('list_of_available_products_Rerun_2000.csv')

In [84]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_2000 = pd.DataFrame(list_of_unavailable_products_Rerun_2000)

list_of_unavailable_products_Rerun_2000.to_csv('list_of_unavailable_products_Rerun_2000.csv')

In [82]:
type(list_of_available_products_Rerun_2000)

pandas.core.frame.DataFrame

In [89]:
list_of_unavailable_products_4000[:10]

['B000PHRIQM',
 'B000PHVNE0',
 'B000PHZ26K',
 'B000PHZE7M',
 'B000PI3BS0',
 'B000PI4J3G',
 'B000PI57HS',
 'B000PI9MO2',
 'B000PK4XRQ',
 'B000PKUYYW']

In [87]:
list_of_unavailable_products_4000=list_of_unavailable_products_4000.values.tolist()

In [88]:
list_of_unavailable_products_4000=oneDArray(list_of_unavailable_products_4000)

In [90]:
list_of_available_products_Rerun_4000 = []
list_of_unavailable_products_Rerun_4000 =[]


#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_4000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_4000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_4000.append(i)
        print('not available',i)
        


https://www.amazon.com/gp/product/B000PHRIQM
not available B000PHRIQM
https://www.amazon.com/gp/product/B000PHVNE0
not available B000PHVNE0
https://www.amazon.com/gp/product/B000PHZ26K
not available B000PHZ26K
https://www.amazon.com/gp/product/B000PHZE7M
not available B000PHZE7M
https://www.amazon.com/gp/product/B000PI3BS0
not available B000PI3BS0
https://www.amazon.com/gp/product/B000PI4J3G
not available B000PI4J3G
https://www.amazon.com/gp/product/B000PI57HS
not available B000PI57HS
https://www.amazon.com/gp/product/B000PI9MO2
not available B000PI9MO2
https://www.amazon.com/gp/product/B000PK4XRQ
not available B000PK4XRQ
https://www.amazon.com/gp/product/B000PKUYYW
not available B000PKUYYW
https://www.amazon.com/gp/product/B000PKUZ5U
not available B000PKUZ5U
https://www.amazon.com/gp/product/B000PKWAWQ
not available B000PKWAWQ
https://www.amazon.com/gp/product/B000PKYBXM
not available B000PKYBXM
https://www.amazon.com/gp/product/B000PLS9YS
not available B000PLS9YS
https://www.amazon.c

not available B000SIS5JW
https://www.amazon.com/gp/product/B000SJ9G0I
not available B000SJ9G0I
https://www.amazon.com/gp/product/B000SQ92RI
not available B000SQ92RI
https://www.amazon.com/gp/product/B000SQRMOS
not available B000SQRMOS
https://www.amazon.com/gp/product/B000SRVQ12
not available B000SRVQ12
https://www.amazon.com/gp/product/B000SRVRAM
not available B000SRVRAM
https://www.amazon.com/gp/product/B000SSDTOI
not available B000SSDTOI
https://www.amazon.com/gp/product/B000SSDTOS
not available B000SSDTOS
https://www.amazon.com/gp/product/B000SSNSAS
not available B000SSNSAS
https://www.amazon.com/gp/product/B000SWY02O
not available B000SWY02O
https://www.amazon.com/gp/product/B000SZVQTQ
not available B000SZVQTQ
https://www.amazon.com/gp/product/B000T217KG
not available B000T217KG
https://www.amazon.com/gp/product/B000T5KATM
not available B000T5KATM
https://www.amazon.com/gp/product/B000T9BMBS
not available B000T9BMBS
https://www.amazon.com/gp/product/B000T9ZCL4
not available B000T9

not available B000VON4NK
https://www.amazon.com/gp/product/B000VON5F2
not available B000VON5F2
https://www.amazon.com/gp/product/B000VON818
not available B000VON818
https://www.amazon.com/gp/product/B000VONB56
not available B000VONB56
https://www.amazon.com/gp/product/B000VOPALY
not available B000VOPALY
https://www.amazon.com/gp/product/B000VP9BWW
not available B000VP9BWW
https://www.amazon.com/gp/product/B000VPG5EE
not available B000VPG5EE
https://www.amazon.com/gp/product/B000VQEQXA
not available B000VQEQXA
https://www.amazon.com/gp/product/B000VT2GYS
not available B000VT2GYS
https://www.amazon.com/gp/product/B000VTQCIY
adding to dictionary B000VTQCIY
https://www.amazon.com/gp/product/B000VTRJMM
not available B000VTRJMM
https://www.amazon.com/gp/product/B000VTVA7M
not available B000VTVA7M
https://www.amazon.com/gp/product/B000VTVD6A
not available B000VTVD6A
https://www.amazon.com/gp/product/B000VUO8VG
not available B000VUO8VG
https://www.amazon.com/gp/product/B000VUT39S
not available

not available B000YZGOZQ
https://www.amazon.com/gp/product/B000YZGP4G
not available B000YZGP4G
https://www.amazon.com/gp/product/B000YZJMHS
not available B000YZJMHS
https://www.amazon.com/gp/product/B000YZLP4Q
not available B000YZLP4Q
https://www.amazon.com/gp/product/B000YZM5P4
not available B000YZM5P4
https://www.amazon.com/gp/product/B000Z4NV66
not available B000Z4NV66
https://www.amazon.com/gp/product/B000Z4U52O
not available B000Z4U52O
https://www.amazon.com/gp/product/B000Z61TX6
not available B000Z61TX6
https://www.amazon.com/gp/product/B000Z63V96
not available B000Z63V96
https://www.amazon.com/gp/product/B000Z63YFM
not available B000Z63YFM
https://www.amazon.com/gp/product/B000Z657ZW
not available B000Z657ZW
https://www.amazon.com/gp/product/B000Z65AA4
not available B000Z65AA4
https://www.amazon.com/gp/product/B000Z65DZG
not available B000Z65DZG
https://www.amazon.com/gp/product/B000Z91JAQ
not available B000Z91JAQ
https://www.amazon.com/gp/product/B000Z96JDI
not available B000Z9

not available B0012NWEZI
https://www.amazon.com/gp/product/B0012NX73Q
not available B0012NX73Q
https://www.amazon.com/gp/product/B0012NX8UI
not available B0012NX8UI
https://www.amazon.com/gp/product/B0012Q78IS
not available B0012Q78IS
https://www.amazon.com/gp/product/B0012QD7EC
not available B0012QD7EC
https://www.amazon.com/gp/product/B0012QK3HG
not available B0012QK3HG
https://www.amazon.com/gp/product/B0012QW06I
not available B0012QW06I
https://www.amazon.com/gp/product/B0012QYX18
not available B0012QYX18
https://www.amazon.com/gp/product/B0012R23UK
not available B0012R23UK
https://www.amazon.com/gp/product/B0012RQZ72
not available B0012RQZ72
https://www.amazon.com/gp/product/B0012RSXNG
not available B0012RSXNG
https://www.amazon.com/gp/product/B0012RUKCI
not available B0012RUKCI
https://www.amazon.com/gp/product/B0012RUL40
not available B0012RUL40
https://www.amazon.com/gp/product/B0012RWOB8
not available B0012RWOB8
https://www.amazon.com/gp/product/B0012RXAGG
not available B0012R

...

not available B0018BT9ZM
https://www.amazon.com/gp/product/B0018C0HLQ
not available B0018C0HLQ
https://www.amazon.com/gp/product/B0018C1FKI
not available B0018C1FKI
https://www.amazon.com/gp/product/B0018C6V98
not available B0018C6V98
https://www.amazon.com/gp/product/B0018CBCAG
not available B0018CBCAG
https://www.amazon.com/gp/product/B0018COTPG
not available B0018COTPG
https://www.amazon.com/gp/product/B0018CW3IQ
not available B0018CW3IQ
https://www.amazon.com/gp/product/B0018DAUKI
not available B0018DAUKI
https://www.amazon.com/gp/product/B0018LLZ2M
not available B0018LLZ2M
https://www.amazon.com/gp/product/B0018N0E4K
not available B0018N0E4K
https://www.amazon.com/gp/product/B0018N5I9Q
not available B0018N5I9Q
https://www.amazon.com/gp/product/B0018NPA9E
not available B0018NPA9E
https://www.amazon.com/gp/product/B0018O07RI
not available B0018O07RI
https://www.amazon.com/gp/product/B0018OVKQ0
not available B0018OVKQ0
https://www.amazon.com/gp/product/B0018QE376
not available B0018Q

not available B001AWG0BK
https://www.amazon.com/gp/product/B001AWG0IS
not available B001AWG0IS
https://www.amazon.com/gp/product/B001AWVR5O
not available B001AWVR5O
https://www.amazon.com/gp/product/B001AXIWGA
not available B001AXIWGA
https://www.amazon.com/gp/product/B001AY8XSQ
not available B001AY8XSQ
https://www.amazon.com/gp/product/B001AZPK90
not available B001AZPK90
https://www.amazon.com/gp/product/B001AZS6R8
not available B001AZS6R8
https://www.amazon.com/gp/product/B001AZXB0A
not available B001AZXB0A
https://www.amazon.com/gp/product/B001B0TLYE
not available B001B0TLYE
https://www.amazon.com/gp/product/B001B0X05U
not available B001B0X05U
https://www.amazon.com/gp/product/B001B153BS
not available B001B153BS
https://www.amazon.com/gp/product/B001B1831U
not available B001B1831U
https://www.amazon.com/gp/product/B001B1MFM8
not available B001B1MFM8
https://www.amazon.com/gp/product/B001B1T2SS
not available B001B1T2SS
https://www.amazon.com/gp/product/B001B2A2LS
not available B001B2

not available B001DHNBYQ
https://www.amazon.com/gp/product/B001DITLI0
not available B001DITLI0
https://www.amazon.com/gp/product/B001DJ2V0E
not available B001DJ2V0E
https://www.amazon.com/gp/product/B001DK4HRS
not available B001DK4HRS
https://www.amazon.com/gp/product/B001DK9RQE
not available B001DK9RQE
https://www.amazon.com/gp/product/B001DKEW6O
not available B001DKEW6O
https://www.amazon.com/gp/product/B001DKF08S
not available B001DKF08S
https://www.amazon.com/gp/product/B001DKGJQU
not available B001DKGJQU
https://www.amazon.com/gp/product/B001DKIFSA
not available B001DKIFSA
https://www.amazon.com/gp/product/B001DKK7Y0
not available B001DKK7Y0
https://www.amazon.com/gp/product/B001DKRM8O
not available B001DKRM8O
https://www.amazon.com/gp/product/B001DKRMB6
not available B001DKRMB6
https://www.amazon.com/gp/product/B001DMPAVI
not available B001DMPAVI
https://www.amazon.com/gp/product/B001DNZJW2
not available B001DNZJW2
https://www.amazon.com/gp/product/B001DO1LRS
not available B001DO

not available B001ET6XKM
https://www.amazon.com/gp/product/B001ET6XMK
not available B001ET6XMK
https://www.amazon.com/gp/product/B001ET6XNO
not available B001ET6XNO
https://www.amazon.com/gp/product/B001ET6XOI
not available B001ET6XOI
https://www.amazon.com/gp/product/B001ET6XVG
not available B001ET6XVG
https://www.amazon.com/gp/product/B001ET6Y4C
not available B001ET6Y4C
https://www.amazon.com/gp/product/B001ET6ZNM
not available B001ET6ZNM
https://www.amazon.com/gp/product/B001ET72C0
not available B001ET72C0
https://www.amazon.com/gp/product/B001ET76C6
not available B001ET76C6
https://www.amazon.com/gp/product/B001ET76E4
not available B001ET76E4
https://www.amazon.com/gp/product/B001ET76EY
not available B001ET76EY
https://www.amazon.com/gp/product/B001ET76FI
not available B001ET76FI
https://www.amazon.com/gp/product/B001ET77Q6
not available B001ET77Q6
https://www.amazon.com/gp/product/B001ET7AUY
not available B001ET7AUY
https://www.amazon.com/gp/product/B001ET7AYA
not available B001ET

not available B001GN1VSK
https://www.amazon.com/gp/product/B001GN5S98
not available B001GN5S98
https://www.amazon.com/gp/product/B001GPNWUI
not available B001GPNWUI
https://www.amazon.com/gp/product/B001GTK8P6
not available B001GTK8P6
https://www.amazon.com/gp/product/B001GTK9SC
not available B001GTK9SC
https://www.amazon.com/gp/product/B001GUUAT4
not available B001GUUAT4
https://www.amazon.com/gp/product/B001GXEOC0
not available B001GXEOC0
https://www.amazon.com/gp/product/B001GZUJ1I
adding to dictionary B001GZUJ1I
https://www.amazon.com/gp/product/B001H0YT5Y
not available B001H0YT5Y
https://www.amazon.com/gp/product/B001H270RQ
not available B001H270RQ
https://www.amazon.com/gp/product/B001H3JQ0E
not available B001H3JQ0E
https://www.amazon.com/gp/product/B001H3ZUKE
not available B001H3ZUKE
https://www.amazon.com/gp/product/B001H54SQ4
not available B001H54SQ4
https://www.amazon.com/gp/product/B001H5QZK6
not available B001H5QZK6
https://www.amazon.com/gp/product/B001H9275Y
not available

In [91]:
print(len(list_of_available_products_Rerun_4000))
print(len(list_of_unavailable_products_Rerun_4000))

40
1158


In [92]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_4000 = pd.DataFrame(list_of_available_products_Rerun_4000)

list_of_available_products_Rerun_4000.to_csv('list_of_available_products_Rerun_4000.csv')

In [93]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_4000 = pd.DataFrame(list_of_unavailable_products_Rerun_4000)

list_of_unavailable_products_Rerun_4000.to_csv('list_of_unavailable_products_Rerun_4000.csv')

In [94]:
list_of_unavailable_products_6000=list_of_unavailable_products_6000.values.tolist()

In [95]:
list_of_unavailable_products_6000=oneDArray(list_of_unavailable_products_6000)

In [96]:
list_of_available_products_Rerun_6000 = []
list_of_unavailable_products_Rerun_6000 =[]


#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_6000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_6000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_6000.append(i)
        print('not available',i)
        


https://www.amazon.com/gp/product/B001HTCYPC
not available B001HTCYPC
https://www.amazon.com/gp/product/B001HU6DZS
not available B001HU6DZS
https://www.amazon.com/gp/product/B001I124CC
not available B001I124CC
https://www.amazon.com/gp/product/B001I2DL00
not available B001I2DL00
https://www.amazon.com/gp/product/B001I6NZ0W
not available B001I6NZ0W
https://www.amazon.com/gp/product/B001I6RW6U
not available B001I6RW6U
https://www.amazon.com/gp/product/B001I6U028
not available B001I6U028
https://www.amazon.com/gp/product/B001I90NM2
not available B001I90NM2
https://www.amazon.com/gp/product/B001I90NN6
not available B001I90NN6
https://www.amazon.com/gp/product/B001I90NNQ
not available B001I90NNQ
https://www.amazon.com/gp/product/B001I9M91U
not available B001I9M91U
https://www.amazon.com/gp/product/B001IACZEK
not available B001IACZEK
https://www.amazon.com/gp/product/B001IB0NOS
not available B001IB0NOS
https://www.amazon.com/gp/product/B001IMYEL0
not available B001IMYEL0
https://www.amazon.c

not available B001M6VK2S
https://www.amazon.com/gp/product/B001M7SIFY
not available B001M7SIFY
https://www.amazon.com/gp/product/B001M9HXDA
not available B001M9HXDA
https://www.amazon.com/gp/product/B001M9YQBW
not available B001M9YQBW
https://www.amazon.com/gp/product/B001MA5RKK
not available B001MA5RKK
https://www.amazon.com/gp/product/B001MA5VUQ
not available B001MA5VUQ
https://www.amazon.com/gp/product/B001MA7QFO
not available B001MA7QFO
https://www.amazon.com/gp/product/B001MA9QQQ
not available B001MA9QQQ
https://www.amazon.com/gp/product/B001MAAG0Q
not available B001MAAG0Q
https://www.amazon.com/gp/product/B001MBLAKA
not available B001MBLAKA
https://www.amazon.com/gp/product/B001MGATUW
not available B001MGATUW
https://www.amazon.com/gp/product/B001MIZBZI
not available B001MIZBZI
https://www.amazon.com/gp/product/B001MIZMLG
not available B001MIZMLG
https://www.amazon.com/gp/product/B001MIZMN4
not available B001MIZMN4
https://www.amazon.com/gp/product/B001MIZMO8
not available B001MI

not available B001R2S8HM
https://www.amazon.com/gp/product/B001R4CLG4
not available B001R4CLG4
https://www.amazon.com/gp/product/B001R5Z7CS
not available B001R5Z7CS
https://www.amazon.com/gp/product/B001R654TS
not available B001R654TS
https://www.amazon.com/gp/product/B001REJFSG
not available B001REJFSG
https://www.amazon.com/gp/product/B001RHZC26
not available B001RHZC26
https://www.amazon.com/gp/product/B001RIRIKE
not available B001RIRIKE
https://www.amazon.com/gp/product/B001RQI512
not available B001RQI512
https://www.amazon.com/gp/product/B001RS8NQW
not available B001RS8NQW
https://www.amazon.com/gp/product/B001RSAM6Q
not available B001RSAM6Q
https://www.amazon.com/gp/product/B001RVFUMY
not available B001RVFUMY
https://www.amazon.com/gp/product/B001RVGVF4
not available B001RVGVF4
https://www.amazon.com/gp/product/B001RVMHMU
not available B001RVMHMU
https://www.amazon.com/gp/product/B001S261Q6
not available B001S261Q6
https://www.amazon.com/gp/product/B001S48HLG
not available B001S4

not available B0021MLVA8
https://www.amazon.com/gp/product/B0022077CK
not available B0022077CK
https://www.amazon.com/gp/product/B0022H6JQI
not available B0022H6JQI
https://www.amazon.com/gp/product/B0022PG9G0
not available B0022PG9G0
https://www.amazon.com/gp/product/B0022PG9QK
not available B0022PG9QK
https://www.amazon.com/gp/product/B00239YCXM
not available B00239YCXM
https://www.amazon.com/gp/product/B0023AQAPO
not available B0023AQAPO
https://www.amazon.com/gp/product/B0023ASBSS
not available B0023ASBSS
https://www.amazon.com/gp/product/B0023AT6R8
not available B0023AT6R8
https://www.amazon.com/gp/product/B0023AV2UM
not available B0023AV2UM
https://www.amazon.com/gp/product/B0023AV2XO
not available B0023AV2XO
https://www.amazon.com/gp/product/B0023AV31U
not available B0023AV31U
https://www.amazon.com/gp/product/B00243FKN8
not available B00243FKN8
https://www.amazon.com/gp/product/B00243FOGG
adding to dictionary B00243FOGG
https://www.amazon.com/gp/product/B0024WDWU2
not available

not available B002BWOY1G
https://www.amazon.com/gp/product/B002BWOY6G
not available B002BWOY6G
https://www.amazon.com/gp/product/B002BWPXFW
not available B002BWPXFW
https://www.amazon.com/gp/product/B002BWPXGQ
not available B002BWPXGQ
https://www.amazon.com/gp/product/B002BWPXH0
not available B002BWPXH0
https://www.amazon.com/gp/product/B002BXGV52
not available B002BXGV52
https://www.amazon.com/gp/product/B002C0338W
not available B002C0338W
https://www.amazon.com/gp/product/B002C0CZEU
not available B002C0CZEU
https://www.amazon.com/gp/product/B002C1JEPM
not available B002C1JEPM
https://www.amazon.com/gp/product/B002C67EN6
not available B002C67EN6
https://www.amazon.com/gp/product/B002C74884
not available B002C74884
https://www.amazon.com/gp/product/B002C794DS
not available B002C794DS
https://www.amazon.com/gp/product/B002C7EYUQ
not available B002C7EYUQ
https://www.amazon.com/gp/product/B002C8A0FI
not available B002C8A0FI
https://www.amazon.com/gp/product/B002CEV1W8
not available B002CE

...

not available B002PMLGOU
https://www.amazon.com/gp/product/B002PMLGVI
not available B002PMLGVI
https://www.amazon.com/gp/product/B002PMWPD6
not available B002PMWPD6
https://www.amazon.com/gp/product/B002PNBZIQ
not available B002PNBZIQ
https://www.amazon.com/gp/product/B002Q52YKG
not available B002Q52YKG
https://www.amazon.com/gp/product/B002Q6UAH4
not available B002Q6UAH4
https://www.amazon.com/gp/product/B002Q6YYRG
not available B002Q6YYRG
https://www.amazon.com/gp/product/B002QA3130
not available B002QA3130
https://www.amazon.com/gp/product/B002QA52V4
not available B002QA52V4
https://www.amazon.com/gp/product/B002QANC2A
not available B002QANC2A
https://www.amazon.com/gp/product/B002QASVFS
not available B002QASVFS
https://www.amazon.com/gp/product/B002QAV560
not available B002QAV560
https://www.amazon.com/gp/product/B002QBBMVM
not available B002QBBMVM
https://www.amazon.com/gp/product/B002QF4UYY
not available B002QF4UYY
https://www.amazon.com/gp/product/B002QFMY42
not available B002QF

not available B002VA4FX0
https://www.amazon.com/gp/product/B002VA4LMU
not available B002VA4LMU
https://www.amazon.com/gp/product/B002VAZINQ
not available B002VAZINQ
https://www.amazon.com/gp/product/B002VBJUY8
not available B002VBJUY8
https://www.amazon.com/gp/product/B002VBRPZO
not available B002VBRPZO
https://www.amazon.com/gp/product/B002VCJHU4
not available B002VCJHU4
https://www.amazon.com/gp/product/B002VCJI76
not available B002VCJI76
https://www.amazon.com/gp/product/B002VCV2AM
not available B002VCV2AM
https://www.amazon.com/gp/product/B002VCXIEK
not available B002VCXIEK
https://www.amazon.com/gp/product/B002VCXS36
not available B002VCXS36
https://www.amazon.com/gp/product/B002VD66RA
not available B002VD66RA
https://www.amazon.com/gp/product/B002VECIE4
not available B002VECIE4
https://www.amazon.com/gp/product/B002VECJ3Y
not available B002VECJ3Y
https://www.amazon.com/gp/product/B002VECJGG
not available B002VECJGG
https://www.amazon.com/gp/product/B002VECJIY
not available B002VE

not available B003156NFW
https://www.amazon.com/gp/product/B003156NJ8
not available B003156NJ8
https://www.amazon.com/gp/product/B003156NMU
not available B003156NMU
https://www.amazon.com/gp/product/B003156OJC
not available B003156OJC
https://www.amazon.com/gp/product/B003156OJM
not available B003156OJM
https://www.amazon.com/gp/product/B003176BES
not available B003176BES
https://www.amazon.com/gp/product/B00317FSMY
not available B00317FSMY
https://www.amazon.com/gp/product/B00317FT4G
not available B00317FT4G
https://www.amazon.com/gp/product/B0031B51Y0
not available B0031B51Y0
https://www.amazon.com/gp/product/B0031DMKVA
not available B0031DMKVA
https://www.amazon.com/gp/product/B0031NK6J8
not available B0031NK6J8
https://www.amazon.com/gp/product/B0031NLHXC
not available B0031NLHXC
https://www.amazon.com/gp/product/B0031NRC86
not available B0031NRC86
https://www.amazon.com/gp/product/B0031SQ5WK
not available B0031SQ5WK
https://www.amazon.com/gp/product/B0031W8Y3E
not available B0031W

not available B00373DIYM
https://www.amazon.com/gp/product/B00373FE76
not available B00373FE76
https://www.amazon.com/gp/product/B00373FPG6
not available B00373FPG6
https://www.amazon.com/gp/product/B00373OK3A
not available B00373OK3A
https://www.amazon.com/gp/product/B00374LSVQ
not available B00374LSVQ
https://www.amazon.com/gp/product/B00375L8SI
not available B00375L8SI
https://www.amazon.com/gp/product/B00378345U
not available B00378345U
https://www.amazon.com/gp/product/B0037BOLVS
not available B0037BOLVS
https://www.amazon.com/gp/product/B0037BQKTO
not available B0037BQKTO
https://www.amazon.com/gp/product/B0037FO44I
not available B0037FO44I
https://www.amazon.com/gp/product/B0037LG6UW
not available B0037LG6UW
https://www.amazon.com/gp/product/B0037LG7L0
not available B0037LG7L0
https://www.amazon.com/gp/product/B0037LKIAG
not available B0037LKIAG
https://www.amazon.com/gp/product/B0037MDBMC
not available B0037MDBMC
https://www.amazon.com/gp/product/B0037MFCHE
not available B0037M

not available B003CIZQGA
https://www.amazon.com/gp/product/B003COAFPQ
not available B003COAFPQ
https://www.amazon.com/gp/product/B003CQZ1A8
not available B003CQZ1A8
https://www.amazon.com/gp/product/B003CT35B2
not available B003CT35B2
https://www.amazon.com/gp/product/B003CW9VCG
not available B003CW9VCG
https://www.amazon.com/gp/product/B003CX2F96
not available B003CX2F96
https://www.amazon.com/gp/product/B003D1YOCS
not available B003D1YOCS
https://www.amazon.com/gp/product/B003D7FE80
not available B003D7FE80
https://www.amazon.com/gp/product/B003D97NY6
not available B003D97NY6
https://www.amazon.com/gp/product/B003DAXHW2
not available B003DAXHW2
https://www.amazon.com/gp/product/B003DGWTXE
not available B003DGWTXE
https://www.amazon.com/gp/product/B003DKO7RG
not available B003DKO7RG
https://www.amazon.com/gp/product/B003DL3YW4
not available B003DL3YW4
https://www.amazon.com/gp/product/B003DQP0O4
not available B003DQP0O4
https://www.amazon.com/gp/product/B003DS0B7I
not available B003DS

In [97]:
print(len(list_of_available_products_Rerun_6000))
print(len(list_of_unavailable_products_Rerun_6000))

12
1207


In [98]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_6000 = pd.DataFrame(list_of_available_products_Rerun_6000)

list_of_available_products_Rerun_6000.to_csv('list_of_available_products_Rerun_6000.csv')

In [99]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_6000 = pd.DataFrame(list_of_unavailable_products_Rerun_6000)

list_of_unavailable_products_Rerun_6000.to_csv('list_of_unavailable_products_Rerun_6000.csv')

In [100]:
list_of_unavailable_products_8000=list_of_unavailable_products_8000.values.tolist()

In [101]:
list_of_unavailable_products_8000=oneDArray(list_of_unavailable_products_8000)

In [ ]:
list_of_available_products_Rerun_8000 = []
list_of_unavailable_products_Rerun_8000 =[]


#Instantiating the web driver
driver = webdriver.Chrome(executable_path="/Users/agarai/Desktop/Sweta-Data_Science/Python_Project1/chromedriver")
#driver = webdriver.Chrome(executable_path="/Users/swetaprabha/Desktop/Job_Preparation/RA_Recommender_System/chromedriver")

#Start looping through first 10000-10500 products in the asin list
for i in list_of_unavailable_products_8000:  
    
    # Print the urls at the beginning of each loop 
    print(f'https://www.amazon.com/gp/product/{i}')
    
    # Send webdriver to the url in the loop
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    
    
    # Instantiate BeautifulSoup web scraper 
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    
    # Create an empty dictionary for each product with asin number i
    product_info = {}
    
    #Try to locate tags to web scrape
    try:
        # Save product ID # so I can identify which product the scrape belongs to 
        product_info['asin'] = i

        # Look for product title 
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        
        # Look for product categories list 
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).findAll('li')[2].text.strip()
         # Look for product description
        product_info['description'] = soup.find('table', {'class':'a-normal a-spacing-micro'}).text.strip()
        
        #Look for Product Price
        product_info['price'] = soup.find('span', {'id':'priceblock_ourprice'}).text.strip() 
        
        #Look for Rating
        product_info['rating']=soup.find('span', {'class':'a-icon-alt'}).text.strip()
        
        #Number of Ratings
        product_info['No_of_Rating']=soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        
        #Add to dictionary
        print('adding to dictionary',i)
        list_of_available_products_Rerun_8000.append(product_info)
        

    except:
        list_of_unavailable_products_Rerun_8000.append(i)
        print('not available',i)
        


https://www.amazon.com/gp/product/B003FKUW3M
not available B003FKUW3M
https://www.amazon.com/gp/product/B003FP86SK
not available B003FP86SK
https://www.amazon.com/gp/product/B003FPE7MY
not available B003FPE7MY
https://www.amazon.com/gp/product/B003FS3L9G
not available B003FS3L9G
https://www.amazon.com/gp/product/B003FSGRW4
not available B003FSGRW4
https://www.amazon.com/gp/product/B003FTXLZY
not available B003FTXLZY
https://www.amazon.com/gp/product/B003FXAOGE
not available B003FXAOGE
https://www.amazon.com/gp/product/B003FXCFZC
not available B003FXCFZC
https://www.amazon.com/gp/product/B003FXE6VI
not available B003FXE6VI
https://www.amazon.com/gp/product/B003G0S7S8
not available B003G0S7S8
https://www.amazon.com/gp/product/B003G0UO48
not available B003G0UO48
https://www.amazon.com/gp/product/B003G1K226
not available B003G1K226
https://www.amazon.com/gp/product/B003GC4GHM
not available B003GC4GHM
https://www.amazon.com/gp/product/B003GCWJFS
not available B003GCWJFS
https://www.amazon.c

not available B003LS7EFC
https://www.amazon.com/gp/product/B003LUFMFO
not available B003LUFMFO
https://www.amazon.com/gp/product/B003LYDBCG
not available B003LYDBCG
https://www.amazon.com/gp/product/B003M09LPK
not available B003M09LPK
https://www.amazon.com/gp/product/B003M1PY2S
not available B003M1PY2S
https://www.amazon.com/gp/product/B003M5UOBK
not available B003M5UOBK
https://www.amazon.com/gp/product/B003MBHVP6
not available B003MBHVP6
https://www.amazon.com/gp/product/B003MBZ2G6
not available B003MBZ2G6
https://www.amazon.com/gp/product/B003MC1ZSE
not available B003MC1ZSE
https://www.amazon.com/gp/product/B003MGRT2Q
not available B003MGRT2Q
https://www.amazon.com/gp/product/B003MRLWF0
not available B003MRLWF0
https://www.amazon.com/gp/product/B003MTPUYC
not available B003MTPUYC
https://www.amazon.com/gp/product/B003MY14J2
not available B003MY14J2
https://www.amazon.com/gp/product/B003N1WQIW
not available B003N1WQIW
https://www.amazon.com/gp/product/B003N1WS9O
not available B003N1

not available B003T2V3FW
https://www.amazon.com/gp/product/B003T4B468
not available B003T4B468
https://www.amazon.com/gp/product/B003T75D9E
not available B003T75D9E
https://www.amazon.com/gp/product/B003TA799S
not available B003TA799S
https://www.amazon.com/gp/product/B003TDZFVE
not available B003TDZFVE
https://www.amazon.com/gp/product/B003TFINPC
not available B003TFINPC
https://www.amazon.com/gp/product/B003TFUGMK
not available B003TFUGMK
https://www.amazon.com/gp/product/B003TG9C74
not available B003TG9C74
https://www.amazon.com/gp/product/B003TJI3EO
not available B003TJI3EO
https://www.amazon.com/gp/product/B003TMO3EU
not available B003TMO3EU
https://www.amazon.com/gp/product/B003TNQG4E
not available B003TNQG4E
https://www.amazon.com/gp/product/B003TNVSTW
not available B003TNVSTW
https://www.amazon.com/gp/product/B003TNY0WE
not available B003TNY0WE
https://www.amazon.com/gp/product/B003TO3UC4
not available B003TO3UC4
https://www.amazon.com/gp/product/B003TO3UG0
not available B003TO

not available B003WHF148
https://www.amazon.com/gp/product/B003WLZXSS
not available B003WLZXSS
https://www.amazon.com/gp/product/B003WM4DXS
not available B003WM4DXS
https://www.amazon.com/gp/product/B003WOVZLY
not available B003WOVZLY
https://www.amazon.com/gp/product/B003WVR9Y4
not available B003WVR9Y4
https://www.amazon.com/gp/product/B003WXJD28
not available B003WXJD28
https://www.amazon.com/gp/product/B003WXSDV0
not available B003WXSDV0
https://www.amazon.com/gp/product/B003WYRNYW
not available B003WYRNYW
https://www.amazon.com/gp/product/B003X18NPW
not available B003X18NPW
https://www.amazon.com/gp/product/B003X4G23O
not available B003X4G23O
https://www.amazon.com/gp/product/B003X4I7HS
not available B003X4I7HS
https://www.amazon.com/gp/product/B003X6EU72
not available B003X6EU72
https://www.amazon.com/gp/product/B003X8749Q
not available B003X8749Q
https://www.amazon.com/gp/product/B003X9E236
not available B003X9E236
https://www.amazon.com/gp/product/B003XFBLVQ
not available B003XF

not available B0042L1XJK
https://www.amazon.com/gp/product/B0042P99WY
not available B0042P99WY
https://www.amazon.com/gp/product/B0042PE8LQ
not available B0042PE8LQ
https://www.amazon.com/gp/product/B0042PEBMM
not available B0042PEBMM
https://www.amazon.com/gp/product/B0042SHWXY
not available B0042SHWXY
https://www.amazon.com/gp/product/B0042SX6RA
not available B0042SX6RA
https://www.amazon.com/gp/product/B0042SXYUY
not available B0042SXYUY
https://www.amazon.com/gp/product/B0042UDBFK
not available B0042UDBFK
https://www.amazon.com/gp/product/B0042WIHNY
not available B0042WIHNY
https://www.amazon.com/gp/product/B0042WJDOQ
not available B0042WJDOQ
https://www.amazon.com/gp/product/B0042XTB5G
not available B0042XTB5G
https://www.amazon.com/gp/product/B0042ZB70Q
not available B0042ZB70Q
https://www.amazon.com/gp/product/B00433BVR6
not available B00433BVR6
https://www.amazon.com/gp/product/B00433FU64
not available B00433FU64
https://www.amazon.com/gp/product/B00433P91A
not available B00433

...

not available B004CCU9K8
https://www.amazon.com/gp/product/B004CDWZMC
not available B004CDWZMC
https://www.amazon.com/gp/product/B004CEJTOS
not available B004CEJTOS
https://www.amazon.com/gp/product/B004CG0XIW
not available B004CG0XIW
https://www.amazon.com/gp/product/B004CHF8AY
not available B004CHF8AY
https://www.amazon.com/gp/product/B004CNRDBU
not available B004CNRDBU
https://www.amazon.com/gp/product/B004CQ710U
not available B004CQ710U
https://www.amazon.com/gp/product/B004CQR9L6
not available B004CQR9L6
https://www.amazon.com/gp/product/B004CQXZ5U
not available B004CQXZ5U
https://www.amazon.com/gp/product/B004CQXZCS
not available B004CQXZCS
https://www.amazon.com/gp/product/B004CQXZD2
not available B004CQXZD2
https://www.amazon.com/gp/product/B004CQXZPA
not available B004CQXZPA
https://www.amazon.com/gp/product/B004CSU8IA
not available B004CSU8IA
https://www.amazon.com/gp/product/B004CSW1QM
not available B004CSW1QM
https://www.amazon.com/gp/product/B004CZ3V8W
not available B004CZ

not available B004GYQLFE
https://www.amazon.com/gp/product/B004H0UGLW
not available B004H0UGLW
https://www.amazon.com/gp/product/B004H2MB5Y
not available B004H2MB5Y
https://www.amazon.com/gp/product/B004H4TWJU
not available B004H4TWJU
https://www.amazon.com/gp/product/B004H53CY0
not available B004H53CY0
https://www.amazon.com/gp/product/B004H6SHYO
not available B004H6SHYO
https://www.amazon.com/gp/product/B004H85FTM
not available B004H85FTM
https://www.amazon.com/gp/product/B004HAI0SI
not available B004HAI0SI
https://www.amazon.com/gp/product/B004HAMIJA
not available B004HAMIJA
https://www.amazon.com/gp/product/B004HCECVA
not available B004HCECVA
https://www.amazon.com/gp/product/B004HCNF1S
not available B004HCNF1S
https://www.amazon.com/gp/product/B004HE94SE
not available B004HE94SE
https://www.amazon.com/gp/product/B004HFBC9W
not available B004HFBC9W
https://www.amazon.com/gp/product/B004HFBDE6
not available B004HFBDE6
https://www.amazon.com/gp/product/B004HFEOEM
not available B004HF

not available B004L5NBSS
https://www.amazon.com/gp/product/B004L81VXW
not available B004L81VXW
https://www.amazon.com/gp/product/B004L8J15C
not available B004L8J15C
https://www.amazon.com/gp/product/B004L938P0
not available B004L938P0
https://www.amazon.com/gp/product/B004L96NZM
not available B004L96NZM
https://www.amazon.com/gp/product/B004L96NZW
not available B004L96NZW
https://www.amazon.com/gp/product/B004L96O10
not available B004L96O10
https://www.amazon.com/gp/product/B004LC4MHK
not available B004LC4MHK
https://www.amazon.com/gp/product/B004LE0YW0
not available B004LE0YW0
https://www.amazon.com/gp/product/B004LEFDJE
not available B004LEFDJE
https://www.amazon.com/gp/product/B004LEJ8R2
not available B004LEJ8R2
https://www.amazon.com/gp/product/B004LGX7XG
not available B004LGX7XG
https://www.amazon.com/gp/product/B004LIG6SC
not available B004LIG6SC
https://www.amazon.com/gp/product/B004LJ0ZK6
not available B004LJ0ZK6
https://www.amazon.com/gp/product/B004LQGQ48
not available B004LQ

not available B004QLOFS2
https://www.amazon.com/gp/product/B004QLRW82
not available B004QLRW82
https://www.amazon.com/gp/product/B004QRX6U4
not available B004QRX6U4
https://www.amazon.com/gp/product/B004QWD9QA
not available B004QWD9QA
https://www.amazon.com/gp/product/B004QWRCWM
not available B004QWRCWM
https://www.amazon.com/gp/product/B004QX68SK
not available B004QX68SK
https://www.amazon.com/gp/product/B004R7PP3O
not available B004R7PP3O
https://www.amazon.com/gp/product/B004R7R8G6
not available B004R7R8G6
https://www.amazon.com/gp/product/B004RBZ9MM
not available B004RBZ9MM
https://www.amazon.com/gp/product/B004RC4FSK
not available B004RC4FSK
https://www.amazon.com/gp/product/B004RDP6GE
not available B004RDP6GE
https://www.amazon.com/gp/product/B004RDSISC
not available B004RDSISC
https://www.amazon.com/gp/product/B004RDSITG
not available B004RDSITG
https://www.amazon.com/gp/product/B004RDUILC
not available B004RDUILC
https://www.amazon.com/gp/product/B004REGO00
not available B004RE

not available B004WMFV6K
https://www.amazon.com/gp/product/B004WO9EDY
not available B004WO9EDY
https://www.amazon.com/gp/product/B004WP28UE
not available B004WP28UE
https://www.amazon.com/gp/product/B004WPGXMI
not available B004WPGXMI
https://www.amazon.com/gp/product/B004WPHFE8
not available B004WPHFE8
https://www.amazon.com/gp/product/B004WPHT1M
not available B004WPHT1M
https://www.amazon.com/gp/product/B004WSXD4G
not available B004WSXD4G
https://www.amazon.com/gp/product/B004X1P91C
not available B004X1P91C
https://www.amazon.com/gp/product/B004X1QCC2
not available B004X1QCC2
https://www.amazon.com/gp/product/B004X1QECA
not available B004X1QECA
https://www.amazon.com/gp/product/B004X1QM9K
not available B004X1QM9K
https://www.amazon.com/gp/product/B004X1QY20
not available B004X1QY20
https://www.amazon.com/gp/product/B004X1ZUZM
not available B004X1ZUZM
https://www.amazon.com/gp/product/B004X42FES
not available B004X42FES
https://www.amazon.com/gp/product/B004X8IFUW
not available B004X8

In [ ]:
print(len(list_of_available_products_Rerun_8000))
print(len(list_of_unavailable_products_Rerun_8000))

In [ ]:
#Change the list to pandas dataframe to write onto a csv file
list_of_available_products_Rerun_8000 = pd.DataFrame(list_of_available_products_Rerun_8000)

list_of_available_products_Rerun_8000.to_csv('list_of_available_products_Rerun_8000.csv')

In [ ]:
#Change the list to pandas dataframe to write onto a csv file
list_of_unavailable_products_Rerun_8000 = pd.DataFrame(list_of_unavailable_products_Rerun_8000)

list_of_unavailable_products_Rerun_8000.to_csv('list_of_unavailable_products_Rerun_8000.csv')